In [9]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
from tqdm import tqdm

In [ ]:
#run the required sub-functions 
%run WaterBalance_MonthlyFlow.ipynb
%run CC_DailyFlow_Historical_Northcoast.ipynb
%run CC_DailyFlow_Historical_Rev.ipynb
%run CC_DailyFlow.ipynb
%run HydrologicCondition.ipynb

In [2]:
def HCPHydroDFG5Base_Project2020_CC_AdultRev_UMass(myFiles):

    ##### Function computes plausible climate change hydrology for Santa Cruz  
    # Model Developers: Shawn Chartrand (Original MATLAB version)
    #                   Sam Anderson (Translation from MATLAB into Python)
    
    # NOTE: This model version was developed to allow for hydrologic time
    #   to be loaded directly and to bypass associated calculations of flow.
    
    track = 1
    
    # This is a flag which tells the model to use to load up certain
    # input files related to dates used by the model. Historical (1)
    # or future (2) CC scenarios
    HIST = 1
    HIST = 2
    
    # This is a flag which tells the model to use to load up certain
    # input files and set index years depending on whether we are simulating
    # historical (1) or future (2) CC scenarios
    COND = 1
    COND = 2
    
    # This is a flag which tells the model to set-up the Analog Years
    # when we are simulating future (2) CC scenarios or historical (1)
    # does nothing
    ACOND = 1
    ACOND = 2
    
    # This is a flag which tells the model to use historical daily flows (1) or
    # to estimate daily flows using the hydrologic condition (2)
    # FFLOW = 1
    FFLOW = 2
    
    # Control plotting of Figure 1
    PLOT = 1
    
    # This is where the climate realizations are read in one realization at a
    # time and then input to the hydrology calculations
    myDir = ('./Input_Historical/')
    # Gets all txt files in struct
    
    steps = len(myFiles)
    
    # Runtime indication
    print('Initializing...')
    # Initialize the progress bar using tqdm
    with tqdm(total = steps, file=sys.stdout) as progress_bar:
    
        if HIST == 1:
            
            DData1 = pd.read_csv('./Input_Climate/Dates/Dates_Hist.txt', skiprows=0)
            DData = DData1
            # Dates = DData.iloc[:, 0].tolist()
            Dates = DData.iloc[:, 0].to_numpy()
            
        else:
            
            DData1 = pd.read_csv('./Input_Climate/Dates/Dates.txt', skiprows=0)
            DData = DData1
            # Dates = DData.iloc[:, 0].tolist()
            Dates = DData.iloc[:, 0].to_numpy()
            
        for kk in range(steps):
            
            baseFileName = myFiles[kk]
            fullFileName = os.path.join(myDir, baseFileName)
            print('\nNow reading: \n\t', fullFileName)
            # with open(file_full_path, 'r') as f:
            # data = f.read()
            # open(fullFileName)  # Open the file
    
            # Split the filename into it's parts for creating a directory below
            # when writing output to file
            f = os.path.splitext(baseFileName)[0]
            WriteFileName = os.path.join(f)
    
            if ACOND == 2:
                
                # Split the filename to find the climate conditions for use in
                # assigning or calculating the hydrologic condition. This is a
                # bunch of steps.
                
                underlineLocations = [i for i, char in enumerate(baseFileName) if char == '_']
                dotLocations = [i for i, char in enumerate(baseFileName) if char == '.']
                dT_t = baseFileName[underlineLocations[1] + 1:underlineLocations[2]]
                dP_t = baseFileName[underlineLocations[2] + 1:underlineLocations[3]]
                
                # This is used below to assign the analog year for the
                # hydrology records compilation
                
                Realization = baseFileName[underlineLocations[3] + 2:underlineLocations[4]]
                dCV_t = baseFileName[underlineLocations[4] + 1:dotLocations[1]]
                equalLocations_dT = [i for i, char in enumerate(dT_t) if char == '=']
                equalLocations_dP = [i for i, char in enumerate(dP_t) if char == '=']
                equalLocations_dCV = [i for i, char in enumerate(dCV_t) if char == '=']
                dT = float(dT_t[equalLocations_dT[0] + 1:])
                dP = float(dP_t[equalLocations_dP[0] + 1:])
                dCV = float(dCV_t[equalLocations_dCV[0] + 1:])
                
            else:
                
                dT = 0
                dP = 100
                dCV = 1
    
            # This is the climate data from the UMass team. Only take columns 3:5
    
            if baseFileName == 'Historical_SIM.csv' or baseFileName[:7] == 'Weather':#_WBM_dT=2_dP=100_R1270_dCV=1.1.csv': #define 
                delimiter = '\t' #tab delimiter in some files
            else:
                delimiter = ','  #comma delimiter in other files
            
            if ACOND == 1:
                
                # Historical
                Inputdata = pd.read_csv(fullFileName, skiprows=0, delimiter = delimiter) #skip first row because it is column titles
                Climatedata = Inputdata.iloc[:, 4:7]
                
            else:
                
                # Future
                Inputdata = pd.read_csv(fullFileName, header = None, delimiter = delimiter) #don't skip first row as there are not column titles
                Climatedata = Inputdata.iloc[:, 0:3]
                
            # Pre-fill an array to hold the AnalogYear varaible
            AnalogYr = np.zeros(max(Climatedata.shape))
    
            if ACOND == 2:
                
                for j in range(max(Climatedata.shape)):
                    
                    if j < 21:
                        
                        if j < 3:
                            
                            # For use in using the historical daily flows
                            AnalogYr[j] = 2000
                            
                        elif (j > 11) and (j < 15):
                            
                            # For use in using the historical daily flows
                            AnalogYr[j] = 2000
                            
                        else:
                            
                            AnalogYr[j] = 2001
                            
                    else:
                        
                        # Use the R____ part of filename to open the correct
                        # basefile with the analog year information
                        
                        filenc = f"Weather_WBM_dT=0_dP=100_R{Realization}_dCV=1.0.csv"
                        myDir_2 = ('./Input_Historical/')
                        fullFileName2 = os.path.join(myDir_2, filenc)
                        Inputdata2 = pd.read_csv(fullFileName2, skiprows=1)
                        Inputdata2 = pd.read_csv(fullFileName2, skiprows=None)
                        t = Inputdata2.iloc[j, 1]
                        t = datetime.strptime(t, '%Y-%m-%d %H:%M:%S')
                        
                        # For use in using the historical daily flows
                        
                        AnalogYr[j] = t.year
                        
            else:
                
                t = Inputdata.iloc[:, 2]
                # For use in using the historical daily flows
                AnalogYr[:] = t
    
            Climatedata = Climatedata.to_numpy()
            
            ##### Section 1. Big Trees and Northcoast Climate Change Hydrology
    
            # Step 1. Compute the monthly total flow in cfs-days
    
            BT_MonthlyFlow, BT_AnnFlow, Precip, AnalogYr = WaterBalance_MonthlyFlow(Dates, Climatedata, AnalogYr, ACOND)
    
            # Step 2. Compute the monthly hydrologic conditions
    
            if FFLOW == 2:
                
                HydroCond_Daily, HydroCond_Daily_FlowRules, HydroCond_Ann, HydroCond_Mon, SLRMOD7, SLRMOD8, SLRMOD7Flows = HydrologicCondition(
                    BT_MonthlyFlow, BT_AnnFlow, COND)
    
            # Step 3. Compute the decomposed mean daily flow record for Big Trees
    
            if FFLOW == 1:
                
                BT, HC_Hgraph, BTDFlows, WY, SC_Climate, HydroCond_Daily, HydroCond_Daily_FlowRules, HydroCond_Ann, HydroCond_Mon, SLRMOD7, SLRMOD8, SLRMOD7Flows = CC_DailyFlow_Historical_Rev(
                    BT_AnnFlow, BT_MonthlyFlow, Precip, AnalogYr, ACOND, dT, dP, dCV)
                
            elif FFLOW == 2:
                
                BT, HC_Hgraph = CC_DailyFlow(BT_MonthlyFlow, HydroCond_Ann, Precip)
    
            fig, ax = plt.subplots()
            ax.plot(BT[:, 3])
            ax.set_yscale('log')
    
            # Read in file specifying the hydrologic condition for CC
            HC = HydroCond_Daily[:, 4]
    
            HCPFlowRules = HydroCond_Daily_FlowRules
    
            # This is the strait USGS gaged data at the Big Trees station. It has not
            # been corrected for the City diversion immediatly upstream making it an
            # impaired record of flow. I am using the impaired record (BT) in the
            # following models because that is what was used in the original
            # development of the models. When I report the "Natural" flow at the Big
            # Trees station (BTUn)I will use the production data to add back the
            # production for the Big Trees station.
            
            # Specify a length array for calculations
            BTlength = len(BT)
            # Give the modeled variables the date stamp
            # [:,0]=Month % [:,1]=Day % [:,2]=Year
            BTUn = np.zeros((BTlength,4))
            SVc = np.zeros((BTlength,3))
            BTUn[:, :3] = BT[:, :3]
            SVc[:, :3] = BT[:, :3]
    
            # Section 1.1 - BTUn unimpaired by the diversion at Felton
            # Calculate BTUn by adding back in the water lost to production at Felton
            # fopen('./Input/FeltonProdCC_CMIP5.txt');
            # FeltonProd = load('./Input/FeltonProdCC_CMIP5.txt');
    
            for i in range(BTlength):
                
                # Note, everything but the historical catalog should reference column
                # 4 (index 3), not column 5 (index 4). I am setting Felton diversion to zero
                # because we have no idea when or if the diversion will be used
                # in the future, and there are no clear historical trends.
                
                BTUn[i, 3] = BT[i, 3] + 0
                
            # Step 3. Compute the decomposed mean daily flow record for Big Trees
    
            if FFLOW == 1:
                
                LGc, MJc, LDc = CC_DailyFlow_Historical_Northcoast(BT_MonthlyFlow, HydroCond_Ann, Precip, AnalogYr,
                                                                   BTDFlows, WY, SC_Climate)
            
            elif FFLOW == 2:
    
                SVc = np.c_[SVc,np.zeros(BTlength)]
    
                # Initialize
                LGc = np.zeros((BTlength,4))
                MJc = np.zeros((BTlength,4))
                LDc = np.zeros((BTlength,4))
                
                # Give the modeled variables the date stamp
                # [:,0]=Month % [:,1]=Day % [:,2]=Year 
                LGc[:, 0:3] = BT[:, 0:3]
                MJc[:, 0:3] = BT[:, 0:3]
                LDc[:, 0:3] = BT[:, 0:3]
                
                # Section 1.2 - San Vicente
                # Calculate the SVm based on BT
                
                for i in range(BTlength):
                    if BT[i, 0] > 10 or BT[i, 0] < 5:
                        SVc[i, 3] = 0.102554 * BT[i, 3] ** 0.916132 + 0.65
                        # Winter Curve with 0.65cfs added to smooth the transions with
                        # the summer curve, which is more accurate at lower flows
                    else:
                        # SVm[i,3] = 0.0821643*BT[i,3] + 0.730404; version1 line
                        # Summer Line, improved skill overall, July 27, 2010
                        SVc[i, 3] = 0.0000001 * BT[i, 3] * BT[i, 3] * BT[i, 3] - 0.0002056 * BT[i, 3] * BT[i, 3] + 0.1382720 * BT[i, 3] - 0.7500000
                
                # Set a floor value for the flows.
                for i in range(len(SVc)):
                    
                    if SVc[i, 3] < 0.01:
                        
                        SVc[i, 3] = 0.01
                        
                # Section 1.3 Upper Laguna
                # Calculate Laguna flows based on San Vicente record.  Revised regression
                # models are tuned to a much higher degree to hydrologic condition.
                
                for i in range(BTlength):
                    
                    if SVc[i, 3] < 1:
                        
                        if HC[i] == 1:
                            
                            LGc[i, 3] = 0.4043 * (SVc[i, 3] ** 0.1109)
                            
                        elif HC[i] == 2:
                            
                            LGc[i, 3] = 0.4043 * (SVc[i, 3] ** 0.1109)
                            
                        elif HC[i] == 3:
                            
                            LGc[i, 3] = 0.5796 * (SVc[i, 3] ** 0.0957)
                            
                        elif HC[i] == 4:
                            
                            LGc[i, 3] = 0.7551 * (SVc[i, 3] ** 0.088)
                            
                        elif HC[i] == 5:
                            
                            LGc[i, 3] = 0.7551 * (SVc[i, 3] ** 0.088)
                            
                    elif (SVc[i, 3] >= 1) and (SVc[i, 3] < 20):
                        
                        if HC[i] == 1:
                            
                            LGc[i, 3] = 0.3987 * (SVc[i, 3] ** 1.1047)
                            
                        elif HC[i] == 2:
                            
                            LGc[i, 3] = 0.3987 * (SVc[i, 3] ** 1.1047)
                            
                        elif HC[i] == 3:
                            
                            LGc[i, 3] = 0.5452 * SVc[i, 3]
                            
                        elif HC[i] == 4:
                            
                            LGc[i, 3] = 0.748 * (SVc[i, 3] ** 0.8944)
                            
                        elif HC[i] == 5:
                            
                            LGc[i, 3] = 0.748 * (SVc[i, 3] ** 0.8944)
                            
                    else:
                        
                        LGc[i, 3] = 0.5452 * SVc[i, 3]
                        
                # Set a floor value for the flows.
                for i in range(len(LGc)):
                    
                    if LGc[i, 3] < 0.01:
                        
                        LGc[i, 3] = 0.01
                        
                # Section 1.4 Upper Majors
                # Calculate Upper Majors based on Laguna.  The regression models have been
                # revised significantly and are dependent upon flow and hydrologic
                # condition.
                
                for i in range(BTlength):
                    
                    if LGc[i, 3] < 1:
                        
                        if HC[i] == 1:
                            
                            MJc[i, 3] = 0.4455 * LGc[i, 3] + 0.5257
                            
                        elif HC[i] == 2:
                            
                            MJc[i, 3] = 0.4455 * LGc[i, 3] + 0.5257
                            
                        elif HC[i] == 3:
                            
                            MJc[i, 3] = 0.5574 * LGc[i, 3] + 0.6361
                            
                        elif HC[i] == 4:
                            
                            MJc[i, 3] = 0.6148 * LGc[i, 3] + 0.8721
                            
                        elif HC[i] == 5:
                            
                            MJc[i, 3] = 0.6148 * LGc[i, 3] + 0.8721
                            
                    elif (LGc[i, 3] >= 1) and (LGc[i, 3] < 10):
                        
                        if HC[i] == 1:
                            
                            MJc[i, 3] = 0.9577 * (LGc[i, 3] ** 0.7601)
                            
                        elif HC[i] == 2:
                            
                            MJc[i, 3] = 0.9577 * (LGc[i, 3] ** 0.7601)
                            
                        elif HC[i] == 3:
                            
                            MJc[i, 3] = 1.1947 * (LGc[i, 3] ** 0.6766)
                            
                        elif HC[i] == 4:
                            
                            MJc[i, 3] = 1.4873 * (LGc[i, 3] ** 0.581)
                            
                        elif HC[i] == 5:
                            
                            MJc[i, 3] = 1.4873 * (LGc[i, 3] ** 0.581)
                            
                    else:
                        
                        MJc[i, 3] = 0.2225 * (LGc[i, 3] ** 1.3962)
                        
                # Set a floor value for the flows.
                for i in range(len(MJc)):
                    
                    if MJc[i, 3] < 0.01:
                        
                        MJc[i, 3] = 0.01
                        
                # Section 1.5 Liddell Spring
                # # Liddell Springflow is modeled from Upper Laguna flow.
                
                for i in range(BTlength):
                    
                    if LGc[i, 3] < 4:
                        
                        LDc[i, 3] = 1.8414 * (LGc[i, 3] ** 0.1325)
                        
                    else:
                        
                        LDc[i, 3] = 1.5252 * (LGc[i, 3] ** 0.2717)
                        
                # Set a floor value for the flows.
                for i in range(len(LDc)):
                    
                    if LDc[i, 3] < 0.01:
                        
                        LDc[i, 3] = 0.01
                        
            # Section 1.6 Upper Newell
            # Calculate the Upper Newell flow NLm based on BT
            # \\World\Pacific\2004\204124 City of Santa Cruz HCP Support\Hydrologic
            # Calculations and Modeling\204124 San Lorenzo\Newell\
            # Model Newell from BT\newell_BT regression work.xls
            # Newell Upper = 0.00906* BT^1.24839 with R2 = 0.9222
            NLc = np.zeros((BTlength,4))
            NLc[:, 0:3] = BT[:, 0:3]
            NLc[:, 3] = 0.00906 * BT[:, 3] ** 1.24839
            
            # Section 1.7 Loch Lomond Dam
            # Convert NLc to LLm (Loch Lomond at the dam.)
            # LLm = NLc * (8.25mi2 / 4.81mi2) Watershed size adjustment factor.
            LLc = np.zeros((BTlength,4))
            LLc[:, 0:3] = NLc[:, 0:3]
            LLc[:, 3] = NLc[:, 3] * (8.25 / 4.81)
            
            # Set a floor value for the flows.
            for i in range(len(LLc)):
                
                if LLc[i, 3] < 0.01:
                    
                    LLc[i, 3] = 0.01
                    
            ###### Section 2. Anadromous Stations
            # Use combined (modeled & gaged) data to model flow in the anadromous reaches
            # Combine modeled anadromous flow with gaged anadromous flow
            # The way I originally calculated the unimpaired flow was to use the Santa
            # Cruz/Balance flow record to correlate the impaired anadromous flow to the
            # upper station flow. I then found the historic production on those streams
            # and added that to the impaired anadromous flow to get unimpaired
            # anadromous flow. The production records are monthly total, and I divided
            # that value my the number of days in the month to get the daily average
            # production. I will repeat that proceedure for the period I was working
            # with previously (WY70-WY85), for consistancy. Outside of that period I
            # don't have production information, so I used recently gaged data to
            # find a model to calculate the unimpaired andromous directly from the
            # upper station record. I refer to this as the new equation.
            
            # Section 2.1 Anadromous Laguna
            # I have developed a more simple prediction of Unimpaired flow at
            # Anadromous Laguna based on corrected flow records for Upper and
            # Anadromous for the period 10/1/2003 - 9/30/2011.  This is different from
            # how unimpaired anadromous laguna was computed for the back-cast model.
            # Given the uncertainty associated with future hydrology, it made more
            # sense to use an average prediction of unimpaired anadromous flow.
    
            LGAm = np.zeros((BTlength,5))
            LGAm[:, 0:3] = BT[:, 0:3]
            LGAm[:, 3] = 1.4004 * LGc[:, 3] ** 1.013
            LGAm[:, 4] = 1
            
            # Start by setting LGAc = LGAm to fill the array
            LGAc = np.copy(LGAm)
            
            # Set a floor value for the flows.
            for i in range(len(LGAc)):
                
                if LGAc[i, 3] < 0.01:
                    
                    LGAc[i, 3] = 0.01
                    
            # Section 2.2 Anadromous Majors
            # Majors Anadromous Modeled
            # As with Laguna Anadromous, I originally calculated the impared Anadromous
            # record based on the Upper station record, then added in historic
            # production. Recently I developed a model for Majors which extimates
            # unimpared anadromous majors streamflow from the upper station streamflow.
    
            MJAm = np.zeros((BTlength,5))
            MJAm[:, 0:3] = BT[:, 0:3]
            MJAm[:, 3] = 0.9278 * MJc[:, 3] ** 1.0911
            MJAm[:, 4] = 1
            
            # Start by setting MJAc = MJAm to fill the array
            MJAc = np.copy(MJAm)
            
            # Set a floor value for the flows.
            for i in range(len(MJAc)):
                
                if MJAc[i, 3] < 0.01:
                    
                    MJAc[i, 3] = 0.01
    
            # Section 2.3 Anadromous Liddell
            # Calculate the LDAm based on SVc. The "Upper" Liddell station is spring-
            # flow dominated and not well correlated with flow in the Anadromous reach.
            # The Q2 method calculates flow at the Anadromous station based on SVc.
            # Because the reference gage is considered unimpared, the Q2 method
            # estimates unimpared flow in the calculated watershed.
            # \\World\Pacific\2004\204124 City of Santa Cruz HCP Support\Hydrologic
            # Calculations and Modeling\Liddell Model\
            # 204146 Liddell Historing testing August08.xls
            # This modeling is based on the Rantz Q2 method. (Unimpared)
            # The Q2 method doesn't accurately predict the contribution from a spring
            # (due to the "hidden" watershed of a spring.) To account for this we add
            # to the Q2 estimate the springflow.
    
            LDAm = np.zeros((BTlength,5))
            LDAm[:, 0:3] = BT[:, 0:3]
            LDAm[:, 3] = 0.5507 * np.exp(0.8614 * LDc[:, 3])
            LDAm[:, 4] = 1
            
            # Start by setting LDAc = LDAm to fill the array.
            LDAc = np.copy(LDAm)
            
            # Set a floor value for the flows.
            for i in range(len(LDAc)):
                
                if LDAc[i, 3] < 0.01:
                    
                    LDAc[i, 3] = 0.01
                    
            # Section 2.4 Anadromous Newell
            # The expected flow at the anadromous reach below the dam (Rio Rancho
            # Bridge) is calculated as a simple fractional increase in watershed area.
            # In this case the adjustment is:
            # (above dam area + dam to bridge area) / (above dam area)
            # (5283.9 acres + 776.4 acres) / 5283.9 acres = 1.147
            NLAm = np.zeros((BTlength,4))
            NLAm = NLc
            NLAm[:, 3] = LLc[:, 3] * 1.147
            
            # Section 2.5 Tait Street Station
            # Tait Anadromous Modeled
            # The modeled flow was corrected for diversions prior to developing the
            # regression model so no corrections post calculation are needed.
            # Two different regressions models are used to enhance low flow
            # performance.
    
            Tm = np.zeros((BTlength,4))
            Tm[:, 0:3] = BT[:, 0:3]
            
            for i in range(BTlength):
                
                if BT[i, 3] < 30:
                    
                    Tm[i, 3] = 0.94378 * (BT[i, 3] ** 1.05578)
                    
                else:
                    
                    Tm[i, 3] = 1.16888 * (BT[i, 3] ** 0.99281)
    
            # There exists USGS Gaged data for much of the historic record.
            # For now I'm just setting Tc = Tm so that the flowgoal code will work
            Tc = np.copy(Tm)
    
            for i in range(len(Tc)):
    
                if Tc[i,3] < 0.01:
    
                    Tc[i,3] = 0.01
    
            # In the gaged record there are occurances where BT>Tait. We believe these
            # are due to errors in the USGS rating curves and not due to flow loss to
            # groundwater between the two stations. In these cases we are making Tait
            # equal to BT. (added model_V8.m on 5/25/11)
    
            for i in range(BTlength):
    
                if BT[i,3] > Tc[i,3]:
    
                    Tc[i,3] = BT[i,3]
    
            # Section 2.6 Write data to files
    
            # Section 2.7 Watershed Contribution
            # --------------- Start of the new work for V7 --------------------------
            # I need to produce the tributary contribution flows in Confluence format
            # so the residual flows in anadromy can be calculated. This is now
            # done below where I write data to file.
    
            ##### Section 3: Implementing HCP rules
            # Use combined anadromous flows to calculate the habitat flow goals
            # Use anadromous flows - flow goals to calculate available flow
            # at the diversions.
    
            # Calculating the Habitat Flow Goals as created by the project biologist:
            # Jeff Hagar.
    
            # These flow goals come in 5 catagories: baseflow, adult migration,
            # spawning, incubation, and smolt out-migration.
            # The baseflow goals vary based on the volume of flow recorded at the BT
            # gage in the prior month. The historic flows were analized and divided
            # into 5 percentile classes from least flow to most flow recorded.
    
            # (Here the language uses "0-20%" as the driest flows & "80-100%" as the 
            # wettest flows. J. Hagar uses language of "0-20% Exceedence Years" being 
            # the wettest flows and "80-100% Exceedence Years" being the driest flows.)
    
            # The three Tiers are based on the availability of new supply, to be
            # determined, and how dry the prior water years have been.
    
            # -------------------------------------------------------------------------
            # Section 3.1 Flow Goals
            # These are the flow goals for the various streams. I'm putting them up
            # here so that they are easily adjusted without having to dig through the 
            # code below.
    
            # Section 3.1.1 San Lorenzo River Below Tait Street
            # Baseflow matrix with 
            # Block 1 (#,0) being 0-20% WY type, Block 2 (#,1) being 20-40% WY type etc
            # The rows are the baseflow for each month, starting with January. 
            # (Month,Block)
            # TaitBf1 is Tier I  -  TaitBf2 is Tier II   -  TaitBf3 is Tier III
            
            # Create arrays with the desired dimensions (3 variables (Tiers) each with 12 rows (months) and 5 columns (Blocks))
            TaitBf1 = np.zeros((12, 5))
            TaitBf2 = np.zeros((12, 5))
            TaitBf3 = np.zeros((12, 5))
            
            # Block 1, 0-20% WY type (driest flows)
            TaitBf1[0, 0] = 3;      TaitBf2[0, 0] = 4;      TaitBf3[0, 0] = 8;   
            TaitBf1[1, 0] = 5;      TaitBf2[1, 0] = 6;      TaitBf3[1, 0] = 8;   
            TaitBf1[2, 0] = 7;      TaitBf2[2, 0] = 8;      TaitBf3[2, 0] = 8;   
            TaitBf1[3, 0] = 5;      TaitBf2[3, 0] = 5;      TaitBf3[3, 0] = 8;   
            TaitBf1[4, 0] = 4;      TaitBf2[4, 0] = 4;      TaitBf3[4, 0] = 8;   
            TaitBf1[5, 0] = 1;      TaitBf2[5, 0] = 3;      TaitBf3[5, 0] = 8;   
            TaitBf1[6, 0] = 1;      TaitBf2[6, 0] = 3;      TaitBf3[6, 0] = 8;   
            TaitBf1[7, 0] = 1;      TaitBf2[7, 0] = 3;      TaitBf3[7, 0] = 8;   
            TaitBf1[8, 0] = 1;      TaitBf2[8, 0] = 3;      TaitBf3[8, 0] = 8;   
            TaitBf1[9, 0] = 1;      TaitBf2[9, 0] = 3;      TaitBf3[9, 0] = 8;   
            TaitBf1[10, 0] = 1;     TaitBf2[10, 0] = 3;     TaitBf3[10, 0] = 8;   
            TaitBf1[11, 0] = 2;     TaitBf2[11, 0] = 2;     TaitBf3[11, 0] = 8;   
            
            # Block 2, 20-40% WY type (dry flows)
            TaitBf1[0, 1] = 8;      TaitBf2[0, 1] = 13;     TaitBf3[0, 1] = 8;   
            TaitBf1[1, 1] = 14.9;   TaitBf2[1, 1] = 14.9;   TaitBf3[1, 1] = 8;   
            TaitBf1[2, 1] = 15.1;   TaitBf2[2, 1] = 15.1;   TaitBf3[2, 1] = 8;   
            TaitBf1[3, 1] = 15.8;   TaitBf2[3, 1] = 15.8;   TaitBf3[3, 1] = 8;   
            TaitBf1[4, 1] = 15;     TaitBf2[4, 1] = 15;     TaitBf3[4, 1] = 8;   
            TaitBf1[5, 1] = 6;      TaitBf2[5, 1] = 6;      TaitBf3[5, 1] = 8;   
            TaitBf1[6, 1] = 2;      TaitBf2[6, 1] = 2;      TaitBf3[6, 1] = 8;   
            TaitBf1[7, 1] = 1;      TaitBf2[7, 1] = 3;      TaitBf3[7, 1] = 8;   
            TaitBf1[8, 1] = 1;      TaitBf2[8, 1] = 3;      TaitBf3[8, 1] = 8;   
            TaitBf1[9, 1] = 1;      TaitBf2[9, 1] = 3;      TaitBf3[9, 1] = 8;   
            TaitBf1[10, 1] = 4;     TaitBf2[10, 1] = 4;     TaitBf3[10, 1] = 8;   
            TaitBf1[11, 1] = 8;     TaitBf2[11, 1] = 13;    TaitBf3[11, 1] = 8;   
    
            # Block 3, 40-60% WY type (normal flows)
            TaitBf1[0, 2] = 15.8;   TaitBf2[0, 2] = 15.8;   TaitBf3[0, 2] = 15.8;   
            TaitBf1[1, 2] = 15.9;   TaitBf2[1, 2] = 15.9;   TaitBf3[1, 2] = 15.9;   
            TaitBf1[2, 2] = 16.3;   TaitBf2[2, 2] = 16.3;   TaitBf3[2, 2] = 16.3;   
            TaitBf1[3, 2] = 17.2;   TaitBf2[3, 2] = 17.2;   TaitBf3[3, 2] = 17.2;   
            TaitBf1[4, 2] = 17.7;   TaitBf2[4, 2] = 17.7;   TaitBf3[4, 2] = 17.7;   
            TaitBf1[5, 2] = 15;     TaitBf2[5, 2] = 16.6;   TaitBf3[5, 2] = 16.6;   
            TaitBf1[6, 2] = 8;      TaitBf2[6, 2] = 10;     TaitBf3[6, 2] = 12.4;   
            TaitBf1[7, 2] = 4;      TaitBf2[7, 2] = 6;      TaitBf3[7, 2] = 9.8;   
            TaitBf1[8, 2] = 5;      TaitBf2[8, 2] = 6;      TaitBf3[8, 2] = 9;   
            TaitBf1[9, 2] = 4;      TaitBf2[9, 2] = 6;      TaitBf3[9, 2] = 9.8;   
            TaitBf1[10, 2] = 4;     TaitBf2[10, 2] = 6;     TaitBf3[10, 2] = 12.5;   
            TaitBf1[11, 2] = 14;    TaitBf2[11, 2] = 14;    TaitBf3[11, 2] = 15.1;   
    
            # Block 4, 60-80% WY type (wet flows)
            TaitBf1[0, 3] = 16.4;   TaitBf2[0, 3] = 16.4;   TaitBf3[0, 3] = 16.4;   
            TaitBf1[1, 3] = 16.7;   TaitBf2[1, 3] = 16.7;   TaitBf3[1, 3] = 16.7;   
            TaitBf1[2, 3] = 17.3;   TaitBf2[2, 3] = 17.3;   TaitBf3[2, 3] = 17.3;   
            TaitBf1[3, 3] = 17.9;   TaitBf2[3, 3] = 17.9;   TaitBf3[3, 3] = 17.9;   
            TaitBf1[4, 3] = 18.2;   TaitBf2[4, 3] = 18.2;   TaitBf3[4, 3] = 18.2;   
            TaitBf1[5, 3] = 18.1;   TaitBf2[5, 3] = 18.1;   TaitBf3[5, 3] = 18.1;   
            TaitBf1[6, 3] = 14;     TaitBf2[6, 3] = 14;     TaitBf3[6, 3] = 15.8;   
            TaitBf1[7, 3] = 9;      TaitBf2[7, 3] = 9;      TaitBf3[7, 3] = 11.9;   
            TaitBf1[8, 3] = 6;      TaitBf2[8, 3] = 6;      TaitBf3[8, 3] = 11.1;   
            TaitBf1[9, 3] = 6;      TaitBf2[9, 3] = 6;      TaitBf3[9, 3] = 11.4;   
            TaitBf1[10, 3] = 10;    TaitBf2[10, 3] = 12;    TaitBf3[10, 3] = 14.1;   
            TaitBf1[11, 3] = 16;    TaitBf2[11, 3] = 16.2;  TaitBf3[11, 3] = 16.2;   
    
            # Block 5, 80-100% WY type (wettest flows)
            TaitBf1[0, 4] = 17.5;   TaitBf2[0, 4] = 17.5;   TaitBf3[0, 4] = 17.5;   
            TaitBf1[1, 4] = 18;     TaitBf2[1, 4] = 18;     TaitBf3[1, 4] = 18;   
            TaitBf1[2, 4] = 18.2;   TaitBf2[2, 4] = 18.2;   TaitBf3[2, 4] = 18.2;   
            TaitBf1[3, 4] = 18.4;   TaitBf2[3, 4] = 18.4;   TaitBf3[3, 4] = 18.4;   
            TaitBf1[4, 4] = 18.5;   TaitBf2[4, 4] = 18.5;   TaitBf3[4, 4] = 18.5;   
            TaitBf1[5, 4] = 18.5;   TaitBf2[5, 4] = 18.5;   TaitBf3[5, 4] = 18.5;   
            TaitBf1[6, 4] = 18.2;   TaitBf2[6, 4] = 18.2;   TaitBf3[6, 4] = 18.2;   
            TaitBf1[7, 4] = 12;     TaitBf2[7, 4] = 12;     TaitBf3[7, 4] = 16.4;   
            TaitBf1[8, 4] = 10;     TaitBf2[8, 4] = 10;     TaitBf3[8, 4] = 13.3;   
            TaitBf1[9, 4] = 8;      TaitBf2[9, 4] = 8;      TaitBf3[9, 4] = 13.3;   
            TaitBf1[10, 4] = 12;    TaitBf2[10, 4] = 12;    TaitBf3[10, 4] = 16.4;   
            TaitBf1[11, 4] = 17.6;  TaitBf2[11, 4] = 17.6;  TaitBf3[11, 4] = 17.6;   
    
            # San Lorenzo River below Tait Street Adult Migration for December & Jan - March
    
            # Initialize
            TaitMig = np.zeros((1,3))
    
            TaitMig[0,0] = 25.2 # Adult Migration Jan-Mar, Dec
            TaitMig[0,1] = 10   # Outmigration, Jan - May
            TaitMig[0,2] = 8    # Minimum rearing flows Jan - May
            FeltonMig = 40      # Minimum adult migration Jan - Mar, Dec
            SLRMinAFlow = 17    # Minimum instream adult migration flow
    
            # TaitMigS applies in the v10_DFG3 model because TaitMigS > baseflow in dry
            # and critically dry years.
    
            # The North Coast streams calculate the WY type each month to set the flow 
            # goals, and they use the same system from BT.
            # Section 3.1.2 Laguna Creek Anadromous Flow Goals
            # Baseflow Matrix
            # LGABf1 is Tier I  -  LGABf2 is Tier II   -  LGABf3 is Tier III
    
            # Initialize
            LGABf1 = np.zeros((12, 5))
            LGABf2 = np.zeros((12, 5))
            LGABf3 = np.zeros((12, 5))
    
            # Block 1, 0-20% WY type (driest flows)       
            LGABf1[0, 0] = 0.6;    LGABf2[0, 0] = 1.1;    LGABf3[0, 0] = 2;   
            LGABf1[1, 0] = 0.9;    LGABf2[1, 0] = 1.0;    LGABf3[1, 0] = 2;   
            LGABf1[2, 0] = 1.2;    LGABf2[2, 0] = 1.1;    LGABf3[2, 0] = 2;   
            LGABf1[3, 0] = 0.4;    LGABf2[3, 0] = 1.2;    LGABf3[3, 0] = 2;   
            LGABf1[4, 0] = 0.4;    LGABf2[4, 0] = 0.8;    LGABf3[4, 0] = 2;   
            LGABf1[5, 0] = 0.3;    LGABf2[5, 0] = 0.6;    LGABf3[5, 0] = 2;   
            LGABf1[6, 0] = 0.1;    LGABf2[6, 0] = 0.3;    LGABf3[6, 0] = 2;   
            LGABf1[7, 0] = 0.1;    LGABf2[7, 0] = 0.2;    LGABf3[7, 0] = 2;   
            LGABf1[8, 0] = 0.1;    LGABf2[8, 0] = 0.2;    LGABf3[8, 0] = 2;   
            LGABf1[9, 0] = 0.4;    LGABf2[9, 0] = 0.6;    LGABf3[9, 0] = 2;   
            LGABf1[10, 0] = 0.4;   LGABf2[10, 0] = 0.8;   LGABf3[10, 0] = 2;   
            LGABf1[11, 0] = 0.6;   LGABf2[11, 0] = 0.9;   LGABf3[11, 0] = 2;   
    
            # Block 2, 20-40% WY type (dry flows)
            LGABf1[0, 1] = 1.4;    LGABf2[0, 1] = 1.4;    LGABf3[0, 1] = 2;   
            LGABf1[1, 1] = 1.5;    LGABf2[1, 1] = 1.9;    LGABf3[1, 1] = 2;   
            LGABf1[2, 1] = 2.1;    LGABf2[2, 1] = 2.1;    LGABf3[2, 1] = 2;   
            LGABf1[3, 1] = 0.8;    LGABf2[3, 1] = 2.0;    LGABf3[3, 1] = 2;   
            LGABf1[4, 1] = 0.7;    LGABf2[4, 1] = 1.7;    LGABf3[4, 1] = 2;   
            LGABf1[5, 1] = 0.5;    LGABf2[5, 1] = 1.1;    LGABf3[5, 1] = 2;   
            LGABf1[6, 1] = 0.3;    LGABf2[6, 1] = 0.4;    LGABf3[6, 1] = 2;   
            LGABf1[7, 1] = 0.2;    LGABf2[7, 1] = 0.3;    LGABf3[7, 1] = 2;   
            LGABf1[8, 1] = 0.2;    LGABf2[8, 1] = 0.4;    LGABf3[8, 1] = 2;   
            LGABf1[9, 1] = 0.6;    LGABf2[9, 1] = 0.7;    LGABf3[9, 1] = 2;   
            LGABf1[10, 1] = 0.9;   LGABf2[10, 1] = 1.0;   LGABf3[10, 1] = 2;   
            LGABf1[11, 1] = 1.0;   LGABf2[11, 1] = 1.1;   LGABf3[11, 1] = 2;   
    
            # Block 3, 40-60% WY type (normal flows)
            LGABf1[0, 2] = 1.4;    LGABf2[0, 2] = 1.4;    LGABf3[0, 2] = 2;   
            LGABf1[1, 2] = 2.7;    LGABf2[1, 2] = 2.7;    LGABf3[1, 2] = 2;   
            LGABf1[2, 2] = 2.7;    LGABf2[2, 2] = 2.7;    LGABf3[2, 2] = 2;   
            LGABf1[3, 2] = 0.9;    LGABf2[3, 2] = 2.8;    LGABf3[3, 2] = 2;   
            LGABf1[4, 2] = 0.9;    LGABf2[4, 2] = 2.6;    LGABf3[4, 2] = 2;   
            LGABf1[5, 2] = 0.7;    LGABf2[5, 2] = 1.7;    LGABf3[5, 2] = 2;   
            LGABf1[6, 2] = 0.3;    LGABf2[6, 2] = 0.4;    LGABf3[6, 2] = 2;   
            LGABf1[7, 2] = 0.2;    LGABf2[7, 2] = 0.4;    LGABf3[7, 2] = 2;   
            LGABf1[8, 2] = 0.5;    LGABf2[8, 2] = 0.5;    LGABf3[8, 2] = 2;   
            LGABf1[9, 2] = 0.8;    LGABf2[9, 2] = 0.8;    LGABf3[9, 2] = 2;   
            LGABf1[10, 2] = 1.1;   LGABf2[10, 2] = 1.1;   LGABf3[10, 2] = 2;   
            LGABf1[11, 2] = 1.2;   LGABf2[11, 2] = 1.3;   LGABf3[11, 2] = 2;   
    
            # Block 4, 60-80% WY type (wet flows)
            LGABf1[0, 3] = 1.9;    LGABf2[0, 3] = 3.2;    LGABf3[0, 3] = 2;   
            LGABf1[1, 3] = 3.6;    LGABf2[1, 3] = 4.6;    LGABf3[1, 3] = 2;   
            LGABf1[2, 3] = 3.4;    LGABf2[2, 3] = 4.6;    LGABf3[2, 3] = 2;   
            LGABf1[3, 3] = 1.6;    LGABf2[3, 3] = 4.1;    LGABf3[3, 3] = 2;   
            LGABf1[4, 3] = 1.4;    LGABf2[4, 3] = 3.5;    LGABf3[4, 3] = 2;   
            LGABf1[5, 3] = 1.0;    LGABf2[5, 3] = 2.4;    LGABf3[5, 3] = 2;   
            LGABf1[6, 3] = 0.6;    LGABf2[6, 3] = 0.6;    LGABf3[6, 3] = 2;   
            LGABf1[7, 3] = 0.3;    LGABf2[7, 3] = 0.7;    LGABf3[7, 3] = 2;   
            LGABf1[8, 3] = 0.4;    LGABf2[8, 3] = 0.9;    LGABf3[8, 3] = 2;   
            LGABf1[9, 3] = 1.0;    LGABf2[9, 3] = 1.0;    LGABf3[9, 3] = 2;   
            LGABf1[10, 3] = 1.2;   LGABf2[10, 3] = 1.4;   LGABf3[10, 3] = 2;   
            LGABf1[11, 3] = 1.4;   LGABf2[11, 3] = 1.6;   LGABf3[11, 3] = 2;   
    
            # Block 5, 80-100% WY type (wettest flows)
            LGABf1[0, 4] = 3.6;    LGABf2[0, 4] = 4.9;    LGABf3[0, 4] = 2;   
            LGABf1[1, 4] = 5.8;    LGABf2[1, 4] = 6.5;    LGABf3[1, 4] = 2;   
            LGABf1[2, 4] = 6.5;    LGABf2[2, 4] = 6.5;    LGABf3[2, 4] = 2;   
            LGABf1[3, 4] = 4.2;    LGABf2[3, 4] = 6.3;    LGABf3[3, 4] = 2;   
            LGABf1[4, 4] = 2.2;    LGABf2[4, 4] = 4.9;    LGABf3[4, 4] = 2;   
            LGABf1[5, 4] = 1.2;    LGABf2[5, 4] = 3.0;    LGABf3[5, 4] = 2;   
            LGABf1[6, 4] = 0.6;    LGABf2[6, 4] = 0.8;    LGABf3[6, 4] = 2;   
            LGABf1[7, 4] = 0.4;    LGABf2[7, 4] = 0.9;    LGABf3[7, 4] = 2;   
            LGABf1[8, 4] = 0.4;    LGABf2[8, 4] = 1.2;    LGABf3[8, 4] = 2;   
            LGABf1[9, 4] = 1.2;    LGABf2[9, 4] = 1.2;    LGABf3[9, 4] = 2;   
            LGABf1[10, 4] = 1.4;   LGABf2[10, 4] = 1.6;   LGABf3[10, 4] = 2;   
            LGABf1[11, 4] = 1.6;   LGABf2[11, 4] = 1.8;   LGABf3[11, 4] = 2;   
    
            # Migration, spawning, incubation, outmigration
            # Define the LGAMig array as a numpy array with dimensions 1x4
            LGAMig = np.zeros((1, 4))
            
            # Assign values to the LGAMig array
            LGAMig[0, 0] = 15.5  # Adult Migration Jan-Mar, Dec
            LGAMig[0, 1] = 9.4   # Spawning, Jan-Mar, Dec
            LGAMig[0, 2] = 4.0   # Incubation, Jan-Mar, Dec
            LGAMig[0, 3] = 3.8   # Outmigration, Jan-May
    
            # Define the minimum instream adult migration flow
            LGMinAFlow = 10.6    # Minimum instream adult migration flow
    
            # Section 3.1.3 Majors Creek Anadromous Flow Goals
            # Baseflow Matrix
    
            # Initialize arrays for MJABf1, MJABf2, and MJABf3
            MJABf1 = np.zeros((12, 5))
            MJABf2 = np.zeros((12, 5))
            MJABf3 = np.zeros((12, 5))
    
            # Block 1, 0 - 20% WY type (driest flows)
            MJABf1[0, 0] = 0.1;    MJABf2[0, 0] = 0.1;    MJABf3[0, 0] = 0.25;   
            MJABf1[1, 0] = 0.2;    MJABf2[1, 0] = 0.2;    MJABf3[1, 0] = 0.25;   
            MJABf1[2, 0] = 0.3;    MJABf2[2, 0] = 0.3;    MJABf3[2, 0] = 0.25;   
            MJABf1[3, 0] = 0.1;    MJABf2[3, 0] = 0.1;    MJABf3[3, 0] = 0.25;   
            MJABf1[4, 0] = 0.1;    MJABf2[4, 0] = 0.1;    MJABf3[4, 0] = 0.25;   
            MJABf1[5, 0] = 0.1;    MJABf2[5, 0] = 0.1;    MJABf3[5, 0] = 0.25;   
            MJABf1[6, 0] = 0.1;    MJABf2[6, 0] = 0.1;    MJABf3[6, 0] = 0.25;   
            MJABf1[7, 0] = 0.1;    MJABf2[7, 0] = 0.1;    MJABf3[7, 0] = 0.25;   
            MJABf1[8, 0] = 0.1;    MJABf2[8, 0] = 0.1;    MJABf3[8, 0] = 0.25;   
            MJABf1[9, 0] = 0.1;    MJABf2[9, 0] = 0.1;    MJABf3[9, 0] = 0.25;   
            MJABf1[10, 0] = 0.1;   MJABf2[10, 0] = 0.1;   MJABf3[10, 0] = 0.25;   
            MJABf1[11, 0] = 0.1;   MJABf2[11, 0] = 0.1;   MJABf3[11, 0] = 0.25;   
    
            # Block 2, 20 - 40% WY type (dry flows)
            MJABf1[0, 1] = 0.2;    MJABf2[0, 1] = 0.2;    MJABf3[0, 1] = 0.25;   
            MJABf1[1, 1] = 0.7;    MJABf2[1, 1] = 0.7;    MJABf3[1, 1] = 0.25;   
            MJABf1[2, 1] = 1.0;    MJABf2[2, 1] = 1.0;    MJABf3[2, 1] = 0.25;   
            MJABf1[3, 1] = 0.4;    MJABf2[3, 1] = 0.4;    MJABf3[3, 1] = 0.25;   
            MJABf1[4, 1] = 0.1;    MJABf2[4, 1] = 0.1;    MJABf3[4, 1] = 0.25;   
            MJABf1[5, 1] = 0.1;    MJABf2[5, 1] = 0.1;    MJABf3[5, 1] = 0.25;   
            MJABf1[6, 1] = 0.1;    MJABf2[6, 1] = 0.1;    MJABf3[6, 1] = 0.25;   
            MJABf1[7, 1] = 0.1;    MJABf2[7, 1] = 0.1;    MJABf3[7, 1] = 0.25;   
            MJABf1[8, 1] = 0.1;    MJABf2[8, 1] = 0.1;    MJABf3[8, 1] = 0.25;   
            MJABf1[9, 1] = 0.1;    MJABf2[9, 1] = 0.1;    MJABf3[9, 1] = 0.25;   
            MJABf1[10, 1] = 0.1;   MJABf2[10, 1] = 0.1;   MJABf3[10, 1] = 0.25;   
            MJABf1[11, 1] = 0.1;   MJABf2[11, 1] = 0.1;   MJABf3[11, 1] = 0.25;   
    
            # Block 3, 40 - 60% WY type (normal flows)
            MJABf1[0, 2] = 0.5;    MJABf2[0, 2] = 0.5;    MJABf3[0, 2] = 2.2;   
            MJABf1[1, 2] = 1.4;    MJABf2[1, 2] = 1.4;    MJABf3[1, 2] = 4.1;   
            MJABf1[2, 2] = 1.6;    MJABf2[2, 2] = 1.6;    MJABf3[2, 2] = 2.4;   
            MJABf1[3, 2] = 0.8;    MJABf2[3, 2] = 0.8;    MJABf3[3, 2] = 1.7;   
            MJABf1[4, 2] = 0.6;    MJABf2[4, 2] = 0.6;    MJABf3[4, 2] = 1.4;   
            MJABf1[5, 2] = 0.2;    MJABf2[5, 2] = 0.2;    MJABf3[5, 2] = 1.0;   
            MJABf1[6, 2] = 0.1;    MJABf2[6, 2] = 0.1;    MJABf3[6, 2] = 0.8;   
            MJABf1[7, 2] = 0.1;    MJABf2[7, 2] = 0.1;    MJABf3[7, 2] = 0.7;   
            MJABf1[8, 2] = 0.1;    MJABf2[8, 2] = 0.1;    MJABf3[8, 2] = 0.6;   
            MJABf1[9, 2] = 0.1;    MJABf2[9, 2] = 0.1;    MJABf3[9, 2] = 0.8;   
            MJABf1[10, 2] = 0.1;   MJABf2[10, 2] = 0.1;   MJABf3[10, 2] = 1.1;   
            MJABf1[11, 2] = 0.3;   MJABf2[11, 2] = 0.3;   MJABf3[11, 2] = 1.5;       
            
            # Block 4, 60 - 80% WY type (wet flows)
            MJABf1[0, 3] = 1.0;    MJABf2[0, 3] = 1.0;    MJABf3[0, 3] = 2.7;   
            MJABf1[1, 3] = 2.9;    MJABf2[1, 3] = 2.9;    MJABf3[1, 3] = 3.0;   
            MJABf1[2, 3] = 3.4;    MJABf2[2, 3] = 3.4;    MJABf3[2, 3] = 4.3;   
            MJABf1[3, 3] = 1.8;    MJABf2[3, 3] = 1.8;    MJABf3[3, 3] = 3.1;   
            MJABf1[4, 3] = 1.2;    MJABf2[4, 3] = 1.2;    MJABf3[4, 3] = 1.8;   
            MJABf1[5, 3] = 0.5;    MJABf2[5, 3] = 0.5;    MJABf3[5, 3] = 1.2;   
            MJABf1[6, 3] = 0.1;    MJABf2[6, 3] = 0.1;    MJABf3[6, 3] = 1.0;   
            MJABf1[7, 3] = 0.1;    MJABf2[7, 3] = 0.1;    MJABf3[7, 3] = 0.8;   
            MJABf1[8, 3] = 0.1;    MJABf2[8, 3] = 0.1;    MJABf3[8, 3] = 0.7;   
            MJABf1[9, 3] = 0.1;    MJABf2[9, 3] = 0.1;    MJABf3[9, 3] = 0.9;   
            MJABf1[10, 3] = 0.2;   MJABf2[10, 3] = 0.2;   MJABf3[10, 3] = 1.2;   
            MJABf1[11, 3] = 0.4;   MJABf2[11, 3] = 0.4;   MJABf3[11, 3] = 1.9;       
            
            # Block 5, 80 - 100% WY type (wettest flows)
            MJABf1[0, 4] = 4.5;    MJABf2[0, 4] = 4.5;    MJABf3[0, 4] = 4.1;   
            MJABf1[1, 4] = 6.0;    MJABf2[1, 4] = 6.0;    MJABf3[1, 4] = 4.4;   
            MJABf1[2, 4] = 5.7;    MJABf2[2, 4] = 5.7;    MJABf3[2, 4] = 4.7;   
            MJABf1[3, 4] = 4.9;    MJABf2[3, 4] = 4.9;    MJABf3[3, 4] = 3.2;   
            MJABf1[4, 4] = 2.7;    MJABf2[4, 4] = 2.7;    MJABf3[4, 4] = 2.4;   
            MJABf1[5, 4] = 1.2;    MJABf2[5, 4] = 1.2;    MJABf3[5, 4] = 1.6;   
            MJABf1[6, 4] = 0.6;    MJABf2[6, 4] = 0.6;    MJABf3[6, 4] = 1.1;   
            MJABf1[7, 4] = 0.4;    MJABf2[7, 4] = 0.4;    MJABf3[7, 4] = 0.9;   
            MJABf1[8, 4] = 0.2;    MJABf2[8, 4] = 0.2;    MJABf3[8, 4] = 0.7;   
            MJABf1[9, 4] = 0.4;    MJABf2[9, 4] = 0.4;    MJABf3[9, 4] = 0.8;   
            MJABf1[10, 4] = 0.3;   MJABf2[10, 4] = 0.3;   MJABf3[10, 4] = 1.2;   
            MJABf1[11, 4] = 1.0;   MJABf2[11, 4] = 1.0;   MJABf3[11, 4] = 2.1;   
    
            # Migration, spawning, incubation, outmigration
            # TIER II & III
            
            # Initialize MJAMig array with shape (1, 5) and zeros
            MJAMig = np.zeros((1, 5))
            
            # Assign values to MJAMig array
            MJAMig[0, 0] = 16.0  # Adult Migration Jan-Mar, Dec
            MJAMig[0, 1] = 12.1  # Spawning, Jan-Mar, Dec
            MJAMig[0, 2] = 2.9   # Incubation, Jan-Mar, Dec
            MJAMig[0, 3] = 3.4   # Outmigration, Jan-May
            MJAMig[0, 4] = 0.25  # Minimum rearing flows Jan-May
            
            # Define the minimum instream adult migration flow
            MJMinAFlow = 9.0  # Minimum instream adult migration flow
    
            # Section 3.1.4 Liddell Creek Anadromous Flow Goals
            # Baseflow Matrix
    
            # Initialize LDABf1, LDABf2, LDABf3 arrays with shape (12, 5) and zeros
            LDABf1 = np.zeros((12, 5))
            LDABf2 = np.zeros((12, 5))
            LDABf3 = np.zeros((12, 5))
            
            # Block 1, 0-20% WY type (driest flows)
            LDABf1[0, 0] = 0.7;   LDABf2[0, 0] = 0.7;   LDABf3[0, 0] = 0.25;
            LDABf1[1, 0] = 1.0;   LDABf2[1, 0] = 1.0;   LDABf3[1, 0] = 0.25;
            LDABf1[2, 0] = 1.1;   LDABf2[2, 0] = 1.1;   LDABf3[2, 0] = 0.25;
            LDABf1[3, 0] = 1.0;   LDABf2[3, 0] = 1.0;   LDABf3[3, 0] = 0.25;
            LDABf1[4, 0] = 0.6;   LDABf2[4, 0] = 0.6;   LDABf3[4, 0] = 0.25;
            LDABf1[5, 0] = 0.4;   LDABf2[5, 0] = 0.4;   LDABf3[5, 0] = 0.25;
            LDABf1[6, 0] = 0.3;   LDABf2[6, 0] = 0.3;   LDABf3[6, 0] = 0.25;
            LDABf1[7, 0] = 0.2;   LDABf2[7, 0] = 0.2;   LDABf3[7, 0] = 0.25;
            LDABf1[8, 0] = 0.2;   LDABf2[8, 0] = 0.2;   LDABf3[8, 0] = 0.25;
            LDABf1[9, 0] = 0.2;   LDABf2[9, 0] = 0.2;   LDABf3[9, 0] = 0.25;
            LDABf1[10, 0] = 0.4;  LDABf2[10, 0] = 0.4;  LDABf3[10, 0] = 0.25;
            LDABf1[11, 0] = 0.7;  LDABf2[11, 0] = 0.7;  LDABf3[11, 0] = 0.25;
            
            # Block 2, 20-40% WY type (dry flows)
            LDABf1[0, 1] = 1.1;   LDABf2[0, 1] = 1.1;   LDABf3[0, 1] = 0.25;
            LDABf1[1, 1] = 1.5;   LDABf2[1, 1] = 1.5;   LDABf3[1, 1] = 0.25;
            LDABf1[2, 1] = 2.0;   LDABf2[2, 1] = 2.0;   LDABf3[2, 1] = 0.25;
            LDABf1[3, 1] = 1.7;   LDABf2[3, 1] = 1.7;   LDABf3[3, 1] = 0.25;
            LDABf1[4, 1] = 1.4;   LDABf2[4, 1] = 1.4;   LDABf3[4, 1] = 0.25;
            LDABf1[5, 1] = 0.9;   LDABf2[5, 1] = 0.9;   LDABf3[5, 1] = 0.25;
            LDABf1[6, 1] = 0.6;   LDABf2[6, 1] = 0.6;   LDABf3[6, 1] = 0.25;
            LDABf1[7, 1] = 0.5;   LDABf2[7, 1] = 0.5;   LDABf3[7, 1] = 0.25;
            LDABf1[8, 1] = 0.4;   LDABf2[8, 1] = 0.4;   LDABf3[8, 1] = 0.25;
            LDABf1[9, 1] = 0.4;   LDABf2[9, 1] = 0.4;   LDABf3[9, 1] = 0.25;
            LDABf1[10, 1] = 0.7;  LDABf2[10, 1] = 0.7;  LDABf3[10, 1] = 0.25;
            LDABf1[11, 1] = 0.9;  LDABf2[11, 1] = 0.9;  LDABf3[11, 1] = 0.25;
    
            # Block 3, 40-60% WY type (normal flows)
            LDABf1[0, 2] = 1.4;   LDABf2[0, 2] = 1.4;   LDABf3[0, 2] = 2.9;
            LDABf1[1, 2] = 2.6;   LDABf2[1, 2] = 2.6;   LDABf3[1, 2] = 4.6;
            LDABf1[2, 2] = 2.8;   LDABf2[2, 2] = 2.8;   LDABf3[2, 2] = 3.5;
            LDABf1[3, 2] = 2.3;   LDABf2[3, 2] = 2.3;   LDABf3[3, 2] = 3.0;
            LDABf1[4, 2] = 2.0;   LDABf2[4, 2] = 2.0;   LDABf3[4, 2] = 2.6;
            LDABf1[5, 2] = 1.3;   LDABf2[5, 2] = 1.3;   LDABf3[5, 2] = 2.0;
            LDABf1[6, 2] = 0.9;   LDABf2[6, 2] = 0.9;   LDABf3[6, 2] = 1.6;
            LDABf1[7, 2] = 0.7;   LDABf2[7, 2] = 0.7;   LDABf3[7, 2] = 1.4;
            LDABf1[8, 2] = 0.6;   LDABf2[8, 2] = 0.6;   LDABf3[8, 2] = 1.3;
            LDABf1[9, 2] = 0.6;   LDABf2[9, 2] = 0.6;   LDABf3[9, 2] = 1.5;
            LDABf1[10, 2] = 0.9;  LDABf2[10, 2] = 0.9;  LDABf3[10, 2] = 1.8;
            LDABf1[11, 2] = 1.1;  LDABf2[11, 2] = 1.1;  LDABf3[11, 2] = 2.1;
            
            # Block 4, 60-80% WY type (wet flows)
            LDABf1[0, 3] = 2.1;   LDABf2[0, 3] = 2.1;   LDABf3[0, 3] = 3.6;
            LDABf1[1, 3] = 4.2;   LDABf2[1, 3] = 4.2;   LDABf3[1, 3] = 3.9;
            LDABf1[2, 3] = 4.1;   LDABf2[2, 3] = 4.1;   LDABf3[2, 3] = 4.8;
            LDABf1[3, 3] = 3.8;   LDABf2[3, 3] = 3.8;   LDABf3[3, 3] = 4.3;
            LDABf1[4, 3] = 2.9;   LDABf2[4, 3] = 2.9;   LDABf3[4, 3] = 3.3;
            LDABf1[5, 3] = 1.9;   LDABf2[5, 3] = 1.9;   LDABf3[5, 3] = 2.4;
            LDABf1[6, 3] = 1.2;   LDABf2[6, 3] = 1.2;   LDABf3[6, 3] = 1.9;
            LDABf1[7, 3] = 0.9;   LDABf2[7, 3] = 0.9;   LDABf3[7, 3] = 1.7;
            LDABf1[8, 3] = 0.7;   LDABf2[8, 3] = 0.7;   LDABf3[8, 3] = 1.5;
            LDABf1[9, 3] = 0.7;   LDABf2[9, 3] = 0.7;   LDABf3[9, 3] = 1.5;
            LDABf1[10, 3] = 1.0;  LDABf2[10, 3] = 1.0;  LDABf3[10, 3] = 1.9;
            LDABf1[11, 3] = 2.3;  LDABf2[11, 3] = 1.3;  LDABf3[11, 3] = 2.6;
            
            # Block 5, 80-100% WY type (wettest flows)
            LDABf1[0, 4] = 4.4;   LDABf2[0, 4] = 4.4;   LDABf3[0, 4] = 4.7;
            LDABf1[1, 4] = 5.9;   LDABf2[1, 4] = 5.9;   LDABf3[1, 4] = 5.1;
            LDABf1[2, 4] = 5.8;   LDABf2[2, 4] = 5.8;   LDABf3[2, 4] = 5.2;
            LDABf1[3, 4] = 5.6;   LDABf2[3, 4] = 5.6;   LDABf3[3, 4] = 4.6;
            LDABf1[4, 4] = 4.5;   LDABf2[4, 4] = 4.5;   LDABf3[4, 4] = 4.0;
            LDABf1[5, 4] = 2.9;   LDABf2[5, 4] = 2.9;   LDABf3[5, 4] = 2.9;
            LDABf1[6, 4] = 1.8;   LDABf2[6, 4] = 1.8;   LDABf3[6, 4] = 2.2;
            LDABf1[7, 4] = 1.3;   LDABf2[7, 4] = 1.3;   LDABf3[7, 4] = 1.8;
            LDABf1[8, 4] = 1.0;   LDABf2[8, 4] = 1.0;   LDABf3[8, 4] = 1.6;
            LDABf1[9, 4] = 0.9;   LDABf2[9, 4] = 0.9;   LDABf3[9, 4] = 1.6;
            LDABf1[10, 4] = 1.2;  LDABf2[10, 4] = 1.2;  LDABf3[10, 4] = 1.9;
            LDABf1[11, 4] = 1.9;  LDABf2[11, 4] = 1.9;  LDABf3[11, 4] = 3.0;
    
            # Initialize LDAMig array with shape (1, 5) and zeros
            LDAMig = np.zeros((1, 5))
            
            # Migration, spawning, incubation, outmigration
            # TIER II & III
            LDAMig[0, 0] = 11.3  # Adult Migration Jan-Mar, Dec
            LDAMig[0, 1] = 7.4   # Spawning, Jan-Mar, Dec
            LDAMig[0, 2] = 2.0   # Incubation, Jan-Mar, Dec
            LDAMig[0, 3] = 2.0   # Outmigration, Mar-May
            LDAMig[0, 4] = 0.25  # Minimum rearing flows Jan - May
            
            # Minimum instream adult migration flow
            LDMinAFlow = 4.9
    
            ##### Section 3.2 Calculation of HCP Bypass requirements:
            ##### Section 3.2.3 Tait HCP Flow rules
            # First set everything to the baseflow based on WY type, then go back and
            # fill in migration where applicable.
            # C 1   | C 2  | C 3 | C 4 | C 5  | C 6  | C 7   | C 8 | C9
            # Month |Year  | Day |Bf   | Goal | Dur  | Avail | Nat | Dur
    
            # Start with all WYtype flags as 1 (0-20 percentile) & repeat for the other Tiers
    
            # Initialize: 9 columns, with as many rows as BT
            nrows = np.shape(BT)[0]
            TaitHCP3 = np.zeros((nrows, 9))
            TaitHCP3[:,:3] = BT[:,:3]
            TaitHCP3[:,7] = Tc[:,3]
            TaitHCP3[:,8] = 0
            # Specify the Hydrologic Condition to compute the applicable flow rule
            TaitHCP3[:,5] = HCPFlowRules[:,4]
    
            for i in range(BTlength):
    
                # Fill in the baseflow arrays using the hydrologic condition array
                TaitHCP3[i,3:5] = TaitBf3[int(BT[i,0])-1,int(HCPFlowRules[i,4])-1]
    
            # Build a matrix for the three day smolt rule - March - May
            TaitSmolt = np.zeros((nrows, 9))
            TaitSmolt[:,:3] = BT[:,:3]
            TaitSmolt[:,3] = 0
    
            counter = 0
    
            for i in range(BTlength):
                
                if 2 < TaitSmolt[i, 0] < 6:
                    
                    if TaitSmolt[i, 0] == 3 and TaitSmolt[i, 1] == 1:
                        
                        TaitSmolt[i, 3] = 1
                        counter += 1
                        
                    else:
                        
                        TaitSmolt[i, 3] = 1
                        counter += 1
            
                        if 3 < counter < 8:
                            
                            TaitSmolt[i, 3] = 0
                            
                        elif counter == 8:
                            
                            TaitSmolt[i, 3] = 1
                            counter = 1
            
                    if TaitSmolt[i, 0] == 5 and TaitSmolt[i, 1] == 31:
                        
                        counter = 0
    
            TaitSmoltTriggerCD = TaitSmolt[:,3]
    
            # UNCOMMENT BELOW HERE TO RUN THE MOD7/8 ANALYSIS FOR FLOW AVAILABILITY
            # Load a file specifying the months during which Loch Lomond Lake levels drop
            # below the CDFW MOD7 criteria.  This data comes from the spreadsheet titled
            # 'SmoltTriggerCD'.  Loch Lomand data comes from Gary Fiske.  This data
            # covers the non-infrasturcture scenario.
            # with open('./Input/SLRMOD7_CCWarmerTemps_Project_AdultRev2.txt', 'r') as file:
            #     SLRMOD7 = np.loadtxt(file)
    
            # Load a file specifying the 3-days per week of months during which Loch Lomond Lake 
            # levels drop below the CDFW MOD7 criteria.  This data comes from the spreadsheet 
            # titled 'SmoltTriggerCD'.  Loch Lomand data comes from Gary Fiske.  This data
            # covers the non-infrasturcture scenario.
            # with open('./Input/SLRMOD7Flows_CCWarmerTemps_Project_AdultRev2.txt', 'r') as file:
            #     SLRMOD7Flows = np.loadtxt(file)
    
            # Load a file specifying the months during which Loch Lomond Lake levels drop
            # below the CDFW MOD8 criteria.  This data comes from the spreadsheet titled
            # 'SmoltTriggerCD'.  Loch Lomand data comes from Gary Fiske.  This data
            # covers the non-infrasturcture scenario.
            # with open('./Input/SLRMOD8_CCWarmerTemps_Project_AdultRev2.txt', 'r') as file:
            #     SLRMOD8 = np.loadtxt(file)
    
            # SLR at Tait Street
            # Now set migration flows
            # Migration flows are required when the BT flow meets or exceeds the
            # migration flow during WYtype months that are wet enough. 
            # There are no migration flows for Tier I at Tait. 
            # There are no migration flows for the driest three percentile 
            # month types for Tier II.
            # Tier III has full options for migration flows.
    
            # Tier III Migration flows
            # Adult and smolt migration combined for Jan - Mar
            # For the months December through March
            for i in range(BTlength):
    
                if TaitHCP3[i,0] == 12 or TaitHCP3[i,0] <= 4:
    
                    # ADULT MIGRATION, SMOLT MIGRATION AND REARING
    
                    # For flows > the minimum adult migration flow
                    if TaitHCP3[i,7] >= SLRMinAFlow:
    
                        if SLRMOD7[i,0] == 0 and SLRMOD8[i,0] == 0:
    
                            # If the hydrologic condition is average or wetter it is business as usual
                            if TaitHCP3[i,5] >= 3:
    
                                TaitHCP3[i,4] = TaitMig[0,0]
                                TaitHCP3[i,8] = 100
                                
                            # If the hydrologic condition is dry or critically dry
                            elif TaitHCP3[i,5] == 2:
    
                                # If it is April turn off adult migration
                                if TaitHCP3[i,0] == 4:
    
                                    TaitHCP3[i,4] = TaitMig[0,1]
                                    TaitHCP3[i,8] = 0
    
                                else:
    
                                    TaitHCP3[i,4] = TaitMig[0,0]
                                    TaitHCP3[i,8] = 100
    
                            elif TaitHCP3[i,5] == 1:
    
                                # If it is April, critically dry rule governs
                                if TaitHCP3[i,0] == 4:
                                    
                                    # If the 3-day smolt flow trigger = 1 (trigger smolt flow)
                                    if TaitSmoltTriggerCD[i] == 1:
                                        
                                        TaitHCP3[i,4] = TaitMig[0,1]
                                        TaitHCP3[i,8] = 200
                                        
                                    # If the 3-day smolt flow trigger = 0 (trigger rearing flow)
                                    elif TaitSmoltTriggerCD[i] == 0:
                                        
                                        TaitHCP3[i,4] = TaitMig[0,2]
                                        TaitHCP3[i,8] = 300
                                        
                                else:
                                    
                                    TaitHCP3[i,4] = TaitMig[0,0]
                                    TaitHCP3[i,8] = 100
                                    
                        elif SLRMOD7[i,0] == 1 and SLRMOD8[i,0] == 0:
    
                            if SLRMOD7Flows[i,0] == 1:
    
                                TaitHCP3[i,0] = TaitMig[0,0]
                                TaitHCP3[i,8] = 100
    
                            elif SLRMOD7Flows[i,0] == 0:
                                
                                if TaitHCP3[i,5] >= 2:
    
                                    # Only baseflows in December if not migration
                                    if TaitHCP3[i,0] == 12:
    
                                        TaitHCP3[i,4] = TaitMig[0,2]
                                        TaitHCP3[i,8] = 0
                                        
                                    # All other months
                                    else:
    
                                        if TaitHCP3[i,4] > TaitMig[0,1]:
    
                                            TaitHCP3[i,4] = TaitHCP3[i,4] #### SHAWN??
                                            TaitHCP3[i,8] = 299
                                        
                                        else:
    
                                            TaitHCP3[i,4] = TaitMig[0,1]
                                            TaitHCP3[i,8] = 250
    
                                elif TaitHCP3[i,5] < 2:
    
                                    # Only baseflows in December if not migration
                                    if TaitHCP3[i,0] == 12:
    
                                        TaitHCP3[i,4] = TaitMig[0,2]
                                        TaitHCP3[i,8] = 0
                                        
                                    # All other months
                                    else:
    
                                        # If it is March the critically dry smolt rule kicks in
                                        if TaitHCP3[i,0] == 3 or TaitHCP3[i,0] == 4:
    
                                            # If the 3-day smolt flow trigger = 1 (trigger smolt flow)
                                            if TaitSmoltTriggerCD[i] == 1:
    
                                                TaitHCP3[i,4] = TaitMig[0,1]
                                                TaitHCP3[i,8] = 200
                                                
                                            # If the 3-day smolt flow trigger = 0 (do not trigger smolt flow)
                                            elif TaitSmoltTriggerCD[i] == 0:
    
                                                TaitHCP3[i,4] = TaitMig[0,2]
                                                TaitHCP3[i,8] = 300
    
                                        else:
    
                                            if TaitHCP3[i,4] > TaitMig[0,1]:
    
                                                TaitHCP3[i,4] = TaitHCP3[i,4] ##### SHAWN???
                                                TaitHCP3[i,8] = 299
                                            
                                            else:
    
                                                TaitHCP3[i,4] = TaitMig[0,1] ##### SHAWN???
                                                TaitHCP3[i,8] = 250
    
                        elif SLRMOD7[i,0] == 0 and SLRMOD8[i,0] == 1:
    
                            # If a storm occurs trigger MOD8
                            if (TaitHCP3[i,7]/TaitHCP3[i-1,7]) > 1.25  and TaitHCP3[i+1,7] > SLRMinAFlow:
    
                                if TaitHCP3[i,5] > 2:
    
                                    if TaitHCP3[i-1,8] == 0:
    
                                        TaitHCP3[i,4] = TaitMig[0,0]
                                        TaitHCP3[i,8] = 101
    
                                    elif TaitHCP3[i-1,8] >= 101 and TaitHCP3[i-1,8] <= 104:
    
                                        TaitHCP3[i,4] = TaitMig[0,0]
                                        TaitHCP3[i,8] = TaitHCP3[i-1,8] + 1
    
                                    elif TaitHCP3[i-1,8] >= 105 and TaitHCP3[i-1,8] <= 300:
    
                                        TaitHCP3[i,4] = TaitMig[0,0]
                                        TaitHCP3[i,8] = 101
    
                                elif TaitHCP3[i,5] <= 2:
    
                                    if TaitHCP3[i,0] == 12 or TaitHCP3[i,0] == 4:
    
                                        TaitHCP3[i,4] = TaitMig[0,2]
                                        TaitHCP3[i,8] = 0
    
                                    else:
    
                                        if TaitHCP3[i-1,8] == 0:
    
                                            TaitHCP3[i,4] = TaitMig[0,0]
                                            TaitHCP3[i,8] = 101
    
                                        elif TaitHCP3[i-1,8] >= 101 and TaitHCP3[i-1,8] <= 104:
    
                                            TaitHCP3[i,4] = TaitMig[0,0]
                                            TaitHCP3[i,8] = TaitHCP3[i-1,8] + 1
    
                                        elif TaitHCP3[i-1,8] >= 105 and TaitHCP3[i-1,8] <= 300:
    
                                            TaitHCP3[i,4] = TaitMig[0,0]
                                            TaitHCP3[i,8] = 101
    
                            # If no storm occurs, all other outcomes are handled
                            else:
    
                                if TaitHCP3[i-1,8] == 0:
    
                                    # No storm
                                    if TaitHCP3[i,0] == 12:
    
                                        TaitHCP3[i,4] = TaitMig[0,2]
                                        TaitHCP3[i,8] = 0
    
                                    else:
    
                                        # New code to address baseflow vs rearing conflict
                                        if TaitHCP3[i,4] > TaitMig[0,1]:
    
                                            TaitHCP3[i,4] = TaitHCP3[i,4] ##### SHAWN???
                                            TaitHCP3[i,8] = 299
    
                                        else:
    
                                            if TaitHCP3[i,5] >= 2:
    
                                                TaitHCP3[i,4] = TaitMig[0,1]
                                                TaitHCP3[i,8] = 250
    
                                            elif TaitHCP3[i,5] < 2:
    
                                                # If it is March or April the critically dry smolt rule kicks in 
                                                if TaitHCP3[i,0] == 3 or TaitHCP3[i,0] == 4:
    
                                                    # If the 3-day smolt flow trigger = 1 (trigger smolt flow)
                                                    if TaitSmoltTriggerCD[i] == 1:
    
                                                        TaitHCP3[i,4] = TaitMig[0,1]
                                                        TaitHCP3[i,8] = 200
    
                                                    # If the 3-day smolt flow trigger = 0 (do not trigger smolt flow)
                                                    elif TaitSmoltTriggerCD[i] == 0:
    
                                                        TaitHCP3[i,4] = TaitMig[0,2]
                                                        TaitHCP3[i,8] = 300
    
                                                # If it is not March, then baseflows prevail
                                                else:
    
                                                    TaitHCP3[i,4] = TaitMig[0,1]
                                                    TaitHCP3[i,8] = 251
    
                                elif TaitHCP3[i-1,8] == 101:
    
                                    TaitHCP3[i,4] = TaitMig[0,0]
                                    TaitHCP3[i,8] = TaitHCP3[i-1,8] + 1
    
                                elif TaitHCP3[i-1,8] >= 102 and TaitHCP3[i-1,8] <= 104:
    
                                    TaitHCP3[i,4] = TaitMig[0,0]
                                    TaitHCP3[i,8] = TaitHCP3[i-1,8] + 1
    
                                elif TaitHCP3[i-1,8] >= 105 and TaitHCP3[i-1,8] <= 300:
    
                                    if TaitHCP3[i,0] == 12:
    
                                        TaitHCP3[i,4] = TaitMig[0,2]
                                        TaitHCP3[i,8] = 0
    
                                    else:
    
                                        # New code to address baseflow vs rearing conflict
                                        if TaitHCP3[i,4] > TaitMig[0,1]:
    
                                            TaitHCP3[i,4] = TaitHCP3[i,4] ##### SHAWN???
                                            TaitHCP3[i,8] = 299
    
                                        else:
    
                                            if TaitHCP3[i,5] >= 2:
    
                                                TaitHCP3[i,4] = TaitMig[0,1]
                                                TaitHCP3[i,8] = 250
    
                                            elif TaitHCP3[i,5] < 2:
    
                                                # If it is March the critically dry smolt rule kicks in
                                                if TaitHCP3[i,0] == 3 or TaitHCP3[i,0] == 4:
    
                                                    # If the 3-day smolt flow trigger = 1 (trigger smolt flow)
                                                    if TaitSmoltTriggerCD[i] == 1:
    
                                                        TaitHCP3[i,4] = TaitMig[0,1]
                                                        TaitHCP3[i,8] = 200
    
                                                    # If the 3-day smolt flow trigger = 0 (do not trigger smolt flow)
                                                    elif TaitSmoltTriggerCD[i] == 0:
    
                                                        TaitHCP3[i,4] = TaitMig[0,2]
                                                        TaitHCP3[i,8] = 300
    
                                                # If it is not March, then baseflows prevail
                                                else:
    
                                                    TaitHCP3[i,4] = TaitMig[0,1]
                                                    TaitHCP3[i,8] = 251
                                    
                    # If flows < the minimum adult migration flow
                    elif TaitHCP3[i,7] < SLRMinAFlow:
    
                        # If it is December
                        if TaitHCP3[i,0] == 12:
    
                            if TaitHCP3[i,5] >= 3:
    
                                TaitHCP3[i,4] = TaitHCP3[i,4] #### SHAWN???
                                TaitHCP3[i,8] = 400
    
                            elif TaitHCP3[i,5] < 3:
    
                                TaitHCP3[i,4] = TaitMig[0,2]
                                TaitHCP3[i,8] = 300
    
                        # If it is January or February
                        elif TaitHCP3[i,0] == 1 or TaitHCP3[i,0] == 2:
    
                            if TaitHCP3[i,5] >= 2:
    
                                # New code to address baseflow vs rearing conflict
                                if TaitHCP3[i,4] > TaitMig[0,1]:
    
                                    TaitHCP3[i,4] = TaitHCP3[i,4]
                                    TaitHCP3[i,8] = 299
    
                                else:
    
                                    TaitHCP3[i,4] = TaitMig[0,1]
                                    TaitHCP3[i,8] = 250
    
                            else:
    
                                TaitHCP3[i,4] = TaitMig[0,2]
                                TaitHCP3[i,8] = 0
    
                        # If it is March or April
                        elif TaitHCP3[i,0] == 3 or TaitHCP3[i,0] == 4:
    
                            if TaitHCP3[i,5] >= 2:
    
                                # New code to address baseflow vs rearing conflict
                                if TaitHCP3[i,4] > TaitMig[0,1]:
    
                                    TaitHCP3[i,4] = TaitHCP3[i,4]
                                    TaitHCP3[i,8] = 299
    
                                else:
    
                                    TaitHCP3[i,4] = TaitMig[0,1]
                                    TaitHCP3[i,8] = 250
    
                            # To deal with the critically dry smolt rule
                            elif TaitHCP3[i,5] < 2:
    
                                # If the 3-day smolt flow trigger = 1 (trigger smolt flow)
                                if TaitSmoltTriggerCD[i] == 1:
    
                                    TaitHCP3[i,4] = TaitMig[0,1]
                                    TaitHCP3[i,8] = 200
    
                                # If the 3-day smolt flow tirgger = 0 (trigger rearing flow)
                                elif TaitSmoltTriggerCD[i] == 0:    
    
                                    TaitHCP3[i,4] = TaitMig[0,2]
                                    TaitHCP3[i,8] = 300
    
                elif TaitHCP3[i,0] == 5:
    
                    # SMOLT MIGRATION
                                    
                    if TaitHCP3[i,5] >= 2:
    
                        # New code to address baseflow vs rearing conflict
                        if TaitHCP3[i,4] > TaitMig[0,1]:
    
                            TaitHCP3[i,4] = TaitHCP3[i,4]
                            TaitHCP3[i,8] = 299
    
                        else:
    
                            TaitHCP3[i,4] = TaitMig[0,1]
                            TaitHCP3[i,8] = 250
    
                    # For May to deal with the critically dry smolt rule
                    elif TaitHCP3[i,5] < 2:
    
                        # If the 3-day smolt flow trigger = 1 (trigger smolt flow)
                        if TaitSmoltTriggerCD[i] == 1:
    
                            TaitHCP3[i,4] = TaitMig[0,1]
                            TaitHCP3[i,8] = 200
                            
                        # If the 3-day smolt flow trigger = 0 (trigger rearing flow)
                        elif TaitSmoltTriggerCD[i] == 0:
    
                            TaitHCP3[i,4] = TaitMig[0,2]
                            TaitHCP3[i,8] = 300
    
            # This piece of code calculates the flow available for diversion at Tait
            for i in range(BTlength):
            
                # If the flow goal (base or migration) is less than the unimpaired flow
                if TaitHCP3[i, 4] < TaitHCP3[i, 7]:
                    
                    # Then the available flow for diversion = unimpaired - goal.
                    TaitHCP3[i, 6] = TaitHCP3[i, 7] - TaitHCP3[i, 4]
            
                # If the flow goal is greater than or equal to the unimpaired flow
                elif TaitHCP3[i, 4] >= TaitHCP3[i, 7]:
                    
                    # Then the available flow for diversion = unimparied * 0.05
                    TaitHCP3[i, 6] = 0
                    # TaitHCP3[i, 6] = TaitHCP3[i, 7] * 0.05
    
            ##### Section 3.3 North Coast HCP Flow Rules
            # NC HCP Flow rules, using the same 5 percentiles based on BT flows.
            # First, set everything to the baseflow based on WY type, then go back and
            # fill in migration where applicable.
            # This is how the N. Coast arrays are set-up, which is different from how
            # the Tait St. array is set-up. This required minor re-coding of the
            # baseflow assignment code.
            # C 1   | C 2  | C 3 | C 4 | C 5 | C 6  | C 7  | C 8
            # Month | Year | Day | Nat | Bf  | Dur  | Goal | Type
    
            # Initialize: 9 columns, with as many rows as BT
            nrows = np.shape(BT)[0]
            LGAHCP3 = np.zeros((nrows, 8))
            MJAHCP3 = np.zeros((nrows, 8))
            LDAHCP3 = np.zeros((nrows, 8))
    
            # Section 3.3.1 North Coast Baseflow HCP Flow Rules
            LGAHCP3[:, :3] = BT[:, :3]
            LGAHCP3[:, 3] = LGAc[:, 3]
            
            MJAHCP3[:, :3] = BT[:, :3]
            MJAHCP3[:, 3] = MJAc[:, 3]
            
            LDAHCP3[:, :3] = BT[:, :3]
            LDAHCP3[:, 3] = LDAc[:, 3]
            
            # Set the counter to zero
            LGAHCP3[:, 5] = 0
            MJAHCP3[:, 5] = 0
            LDAHCP3[:, 5] = 0
            
            # Specify the Hydrologic Condition
            LGAHCP3[:, 7] = HCPFlowRules[:, 4]
            MJAHCP3[:, 7] = HCPFlowRules[:, 4]
            LDAHCP3[:, 7] = HCPFlowRules[:, 4]
            
            for i in range(BTlength):
                LGAHCP3[i, 4] = LGABf3[int(BT[i, 0])-1, int(HCPFlowRules[i, 4])-1]
                MJAHCP3[i, 4] = MJABf3[int(BT[i, 0])-1, int(HCPFlowRules[i, 4])-1]
                LDAHCP3[i, 4] = LDABf3[int(BT[i, 0])-1, int(HCPFlowRules[i, 4])-1]
            
            # Set the 3-day smolt flow equal to the Tait array
            LagunaSmoltTriggerCD = TaitSmoltTriggerCD
    
            ##### LAGUNA CREEK
            # Section 3.3.2 Laguna Non-Baseflow HCP Flow Rules
            # Now I need to add code for when to use the other flow types, i.e.
            # migraion, spawning, incubation, and smolt migration.
            # 
            # The following code was copied from the Excel/Visual Basic code that was
            # used to identify the flow goals in the original work in 2008. The logic
            # should be preserved as I translate the language.
            # ' Macro for determining whethter we should be using Amig, Spawn, or Incub
            #   flows
            # ' These are based on number of days since trigger and flow. 4/25/08 TjB
            # ' The inputs are the prior duration number = dur1 (dur number for day 
            #   before)
            # ' the duration number prior to dur1 = dur2, Adult Migration flow goal = 
            #   goal2
            # ' the Spawning flow goal = goal3, the Rearing flow goal = goal4
            # ' I chose to hard code the required durations for each flow goal catagory
            #   rather than increase
            # ' the number of inputs to the macro. As of the time of original coding 
            #   the Adult Migration
            # ' was to have a max duration of 7 days, the Spawning was to have a max 
            #   duration of 14 days
            # ' and wasn't to start until there had been at least 2 days of Adult 
            #   Migration flows, and
            # ' the Rearing was to have a max duration of 60 days starting at the same 
            #   time as the Spawning.
            # ' The format I'm using for the duration is reflective of using a 
            #   coordinate (i,j) for a variable,
            # ' as I would do for a MatLab program.
            # ' The first number refers to the mode (1 = Adult Mig, 2 = Spawing, 
            #   and 3 = Rearing, 4 = Smolt Migration
            #   Smolt Migration or OutMigration ouccurs follwoing a winter with active 
            #   spawning only.
            # ' The following numbers refer to the count of days in the duration.
            #   For Example: 102 = Adult Migration, Second Day
            #   For Example: 208 = Spawning, 8th day
            #   For Example: 325 = Rearing/Incubation, 25th day
            # Laguna HCP Tier I has no migration flows, so the flow goal is the 
            # baseflow goal. However, because the migration flows are based on the
            # flows unimpaired by the City's diversions, the outcome of following the
            # rules is based on the same flows for all three tiers. In order to handle
            # as much as possible in the fewest loops, I will run all three tiers for
            # Laguna for Adult Migration in a single loop, with the "dur" duration flag
            # going into the Tier I array ONLY.
            
            # Function duration(date1, dur1, dur2, flow, goal)
            # dur1 and dur2 were the duration code of the prior day and two days, here 
            # they will be a column of flags retained for troubleshooting.
            LGAHCP3[:,6] = LGAHCP3[:,4]
    
            ## Laguna HCP Flow Calculations
            for i in range(2, BTlength):
            
                # For the months December through April
                if LGAHCP3[i,0] == 12 or LGAHCP3[i,0] <= 4:
            
                    # ADULT MIGRATION
            
                    # For flows > the minimum adult migration flow
                    if LGAHCP3[i,3] >= LGMinAFlow:
            
                        # If the hydrologic condition is dry or critically dry    
                        if LGAHCP3[i,7] >= 3:
            
                            # If the habitat flow counter equals zero
                            if LGAHCP3[i-1,5] == 0:
                                LGAHCP3[i,5] = 101
                                LGAHCP3[i,6] = LGAMig[0,0]
            
                            # If the habitat flow counter is >= 101 and < 200    
                            elif LGAHCP3[i-1,5] >= 101 and LGAHCP3[i-1,5] < 200:
                                LGAHCP3[i,5] = LGAHCP3[i-1,5] + 1
                                LGAHCP3[i,6] = LGAMig[0,0]
            
                            # If the habitat flow counter is > 200
                            elif LGAHCP3[i-1,5] >= 200:
                                LGAHCP3[i,5] = 101
                                LGAHCP3[i,6] = LGAMig[0,0]
            
                        elif LGAHCP3[i,7] < 3:
            
                            # For the months December through April
                            if LGAHCP3[i,0] == 12:
            
                                LGAHCP3[i,5] = 101
                                LGAHCP3[i,6] = LGAMig[0,1]
            
                            elif LGAHCP3[i,0] == 4:
            
                                LGAHCP3[i,5] = 101
                                LGAHCP3[i,6] = LGAMig[0,1]
            
                            else:
            
                                # If the habitat flow counter equals zero
                                if LGAHCP3[i-1,5] == 0:
                                    LGAHCP3[i,5] = 101
                                    LGAHCP3[i,6] = LGAMig[0,0]
            
                                # If the habitat flow counter is >= 101 and < 200    
                                elif LGAHCP3[i-1,5] >= 101 and LGAHCP3[i-1,5] < 200:
                                    LGAHCP3[i,5] = LGAHCP3[i-1,5] + 1
                                    LGAHCP3[i,6] = LGAMig[0,0]
            
                                # If the habitat flow counter is > 200
                                elif LGAHCP3[i-1,5] >= 200:
                                    LGAHCP3[i,5] = 101
                                    LGAHCP3[i,6] = LGAMig[0,0]
            
                    # Gap in end related to space for comment above
            
                    # SPAWNING, INCUBATION AND SMOLT MIGRATION
            
                    # For flows < the minimum adult migration flow
                    elif LGAHCP3[i,3] < LGMinAFlow:
    
                        # If the habitat flow counter is == 0    
                        if LGAHCP3[i-1,5] == 0:
            
                            # If the habitat counter == 0, i.e. we are not coming off of
                            # a migration and spawning event, and we are in Jan - Mar, 
                            # apply the smolt rule, otherwise apply the rearing flow.
                            # In all cases, habitat flows will be replaced by baseflows
                            # later on if the baseflow rule is larger than the habitat
                            # flow rule.
                            # For the months January through April
                            if LGAHCP3[i,0] == 1 or LGAHCP3[i,0] <= 4:
            
                                # If the water year type is >= 2
                                if LGAHCP3[i,7] >= 2:
    
                                    LGAHCP3[i,5] = 401
                                    LGAHCP3[i,6] = LGAMig[0,3]
            
                            # This If statement is to deal with times when
                            # hydrologic conditions change from dry, or worse,
                            # to average, or better conditions on the first of
                            # the month for the months January through April
            
                            # If the date is January 1st
                            if LGAHCP3[i,0] == 1 and LGAHCP3[i,1] == 1:
            
                                # If cell i-1 WY type is dry or drier and cell i is average or wetter
                                if LGAHCP3[i-1,7] <= 2 and LGAHCP3[i,7] >= 3:
            
                                    less = np.arange(13,0,-1)
                                    z = len(less)
                                    flow = LGAHCP3[i-less,3]
                                    flowgreater = [i for i in range(len(flow)) if flow[i] > 10.6]
                                    # positionmax = max(flowgreater)
                                    if np.size(flowgreater) > 0:
                                        positionmax = max(flowgreater)
                                    else:
                                        positionmax = []
                                    empty = not bool(positionmax)
        
                                    if not empty:
                                    # empty = bool(positionmax)
            
                                    # if empty == 0:
            
                                        LGAHCP3[i,5] = 201 + (z - (positionmax+1))
                                        LGAHCP3[i,6] = LGAMig[0,1] 
            
                            elif LGAHCP3[i,0] == 2 and LGAHCP3[i,1] == 1:
            
                                # If cell i-1 WY type is dry or drier and cell i is average or wetter
                                if LGAHCP3[i-1,7] <= 2 and LGAHCP3[i,7] >= 3:
            
                                    less = np.arange(13,0,-1)
                                    z = len(less)
                                    flow = LGAHCP3[i-less,3]
                                    flowgreater = [i for i in range(len(flow)) if flow[i] > LGMinAFlow]
                                    # positionmax = max(flowgreater)
                                    if np.size(flowgreater) > 0:
                                        positionmax = max(flowgreater)
                                    else:
                                        positionmax = []
                                    empty = not bool(positionmax)
        
                                    if not empty:
                                    # empty = bool(positionmax)
            
                                    # if empty == 0:
            
                                        LGAHCP3[i,5] = 201 + (z - (positionmax+1))
                                        LGAHCP3[i,6] = LGAMig[0,1] 
            
                            elif LGAHCP3[i,0] == 3 and LGAHCP3[i,1] == 1:
            
                                # If cell i-1 WY type is dry or drier and cell i is average or wetter
                                if LGAHCP3[i-1,7] <= 2 and LGAHCP3[i,7] >= 3:
            
                                    less = np.arange(13,0,-1)
                                    z = len(less)
                                    flow = LGAHCP3[i-less,3]
                                    flowgreater = [i for i in range(len(flow)) if flow[i] > LGMinAFlow]
                                    # positionmax = max(flowgreater)
                                    if np.size(flowgreater) > 0:
                                        positionmax = max(flowgreater)
                                    else:
                                        positionmax = []
                                    empty = not bool(positionmax)
        
                                    if not empty:
                                    # empty = bool(positionmax)
            
                                    # if empty == 0:
            
                                        LGAHCP3[i,5] = 201 + (z - (positionmax+1))
                                        LGAHCP3[i,6] = LGAMig[0,1] 
            
                            elif LGAHCP3[i,0] == 4 and LGAHCP3[i,1] == 1: ##### SHAWN???
            
                                # If cell i-1 WY type is dry or drier and cell i is average or wetter
                                if LGAHCP3[i-1,7] <= 2 and LGAHCP3[i,7] >= 3:
            
                                    less = np.arange(13,0,-1)
                                    z = len(less)
                                    flow = LGAHCP3[i-less,3]
                                    flowgreater = [i for i in range(len(flow)) if flow[i] > LGMinAFlow]
                                    # positionmax = max(flowgreater)
                                    if np.size(flowgreater) > 0:
                                        positionmax = max(flowgreater)
                                    else:
                                        positionmax = []
                                    empty = not bool(positionmax)
        
                                    if not empty:
                                    # empty = bool(positionmax)
            
                                    # if empty == 0:
            
                                        LGAHCP3[i,5] = 201 + (z - (positionmax+1))
                                        LGAHCP3[i,6] = LGAMig[0,1] 
            
                        # If the habitat flow counter is >= 101 and < 200   
                        elif LGAHCP3[i-1,5] >= 101 and LGAHCP3[i-1,5] < 200:
                            LGAHCP3[i,5] = 201
                            LGAHCP3[i,6] = LGAMig[0,1]
            
                        # If the habitat flow counter is >= 201 and < 214   
                        elif LGAHCP3[i-1,5] >= 201 and LGAHCP3[i-1,5] < 214:
                            LGAHCP3[i,5] = LGAHCP3[i-1,5] + 1
                            LGAHCP3[i,6] = LGAMig[0,1]
            
                        # If the habitat flow counter is == 214   
                        elif LGAHCP3[i-1,5] == 214:
                            LGAHCP3[i,5] = 301
                            LGAHCP3[i,6] = LGAMig[0,2]
            
                        # If the habitat flow counter is > 300 and < 360   
                        elif LGAHCP3[i-1,5] > 300 and LGAHCP3[i-1,5] < 360:
                            LGAHCP3[i,5] = LGAHCP3[i-1,5] + 1
                            LGAHCP3[i,6] = LGAMig[0,2]
            
                        # If the habitat flow counter = 360   
                        elif LGAHCP3[i-1,5] == 360:
            
                            # For the months January through April
                            if LGAHCP3[i,0] == 1 or LGAHCP3[i,0] <= 4:
            
                                # If the water year type is >= 2
                                if LGAHCP3[i,7] >= 2:
                                    LGAHCP3[i,5] = 401
                                    LGAHCP3[i,6] = LGAMig[0,3]
            
                        # If the habitat flow counter is >= 400   
                        elif LGAHCP3[i-1,5] >= 400:
                            if i == 11022:
                                print('here4')
            
                            # For the months January through April
                            if LGAHCP3[i,0] == 1 or LGAHCP3[i,0] <= 4:
            
                                # If the water year type is >= 2
                                if LGAHCP3[i,7] >= 2:
                                    LGAHCP3[i,5] = LGAHCP3[i-1,5] + 1
                                    LGAHCP3[i,6] = LGAMig[0,3]                
            
                # Advancing to April adult migration flows stop and the rules for spawning and incubation are different, 
                # also need to deal with times when flows exceed the adult migration flows
                elif LGAHCP3[i,0] == 5:
            
                    # SPAWNING, INCUBATION AND SMOLT MIGRATION
            
                    # If the flow is > the minimum adult migration flow  
                    if LGAHCP3[i,3] >= LGMinAFlow:
            
                        # If the habitat flow counter is = 0   
                        if LGAHCP3[i-1,5] == 0:
            
                            # If the water year type is >= 2
                            if LGAHCP3[i,7] >= 2:
                                LGAHCP3[i,5] = 401
                                LGAHCP3[i,6] = LGAMig[0,3]
            
                        # If the habitat flow counter is >= 101 and < 200  
                        elif LGAHCP3[i-1,5] >= 101 and LGAHCP3[i-1,5] < 200:
                            LGAHCP3[i,5] = 201
                            LGAHCP3[i,6] = LGAMig[0,1]  
            
                        # If the habitat flow counter is >= 201 and < 214   
                        elif LGAHCP3[i-1,5] >= 201 and LGAHCP3[i-1,5] < 214:
                            LGAHCP3[i,5] = LGAHCP3[i-1,5] + 1
                            LGAHCP3[i,6] = LGAMig[0,1]
            
                        # If the habitat flow counter = 214   
                        elif LGAHCP3[i-1,5] == 214:
                            LGAHCP3[i,5] = 301
                            LGAHCP3[i,6] = LGAMig[0,2]
            
                        # If the habitat flow counter is > 300 and < 360   
                        elif LGAHCP3[i-1,5] > 300 and LGAHCP3[i-1,5] < 360:
                            LGAHCP3[i,5] = LGAHCP3[i-1,5] + 1
                            LGAHCP3[i,6] = LGAMig[0,2]
            
                        # If the habitat flow counter = 360   
                        elif LGAHCP3[i-1,5] == 360:
            
                            # If the water year type is >= 2
                            if LGAHCP3[i,7] >= 2:
                                LGAHCP3[i,5] = 400
                                LGAHCP3[i,6] = LGAMig[0,3]
            
                        # If the habitat flow counter is >= 400   
                        elif LGAHCP3[i-1,5] >= 400:
            
                            # If the water year type is >= 2
                            if LGAHCP3[i,7] >= 2:
                                LGAHCP3[i,5] = LGAHCP3[i-1,5] + 1
                                LGAHCP3[i,6] = LGAMig[0,3]
            
                    # If the flow is < the minimum adult migration flow  
                    elif LGAHCP3[i,3] < LGMinAFlow:
            
                        # If the habitat flow counter is = 0   
                        if LGAHCP3[i-1,5] == 0:
            
                            # If the water year type is >= 2
                            if LGAHCP3[i,7] >= 2:
                                LGAHCP3[i,5] = 401
                                LGAHCP3[i,6] = LGAMig[0,3]
            
                        # If the habitat flow counter is >= 101 and < 200  
                        elif LGAHCP3[i-1,5] >= 101 and LGAHCP3[i-1,5] < 200:
                            LGAHCP3[i,5] = 201
                            LGAHCP3[i,6] = LGAMig[0,1]  
            
                        # If the habitat flow counter is >= 201 and < 214   
                        elif LGAHCP3[i-1,5] >= 201 and LGAHCP3[i-1,5] < 214:
                            LGAHCP3[i,5] = LGAHCP3[i-1,5] + 1
                            LGAHCP3[i,6] = LGAMig[0,1]
            
                        # If the habitat flow counter = 214   
                        elif LGAHCP3[i-1,5] == 214:
                            LGAHCP3[i,5] = 301
                            LGAHCP3[i,6] = LGAMig[0,2]
            
                        # If the habitat flow counter is > 300 and < 360   
                        elif LGAHCP3[i-1,5] > 300 and LGAHCP3[i-1,5] < 360:
                            LGAHCP3[i,5] = LGAHCP3[i-1,5] + 1
                            LGAHCP3[i,6] = LGAMig[0,2]
            
                        # If the habitat flow counter = 360   
                        elif LGAHCP3[i-1,5] == 360:
            
                            # If the water year type is >= 2
                            if LGAHCP3[i,7] >= 2:
                                LGAHCP3[i,5] = 401
                                LGAHCP3[i,6] = LGAMig[0,3]
            
                        # If the habitat flow counter is >= 400   
                        elif LGAHCP3[i-1,5] >= 400:
            
                            # If the water year type is >= 2
                            if LGAHCP3[i,7] >= 2:
                                LGAHCP3[i,5] = LGAHCP3[i-1,5] + 1
                                LGAHCP3[i,6] = LGAMig[0,3]
            
                # For the months March through May to deal with the critically dry smolt rule
                if LGAHCP3[i,0] >= 3 and LGAHCP3[i,0] <= 5:
            
                    # SMOLT MIGRATION
            
                    # For the month of March and April
                    if LGAHCP3[i,0] == 3 or LGAHCP3[i,0] == 4:
            
                        # if the water year type is critically dry
                        if LGAHCP3[i,7] == 1:
            
                            # Adult migration flow still rules in March, therefore
                            # If the flows are less than the minimum adult migration flow
                            if LGAHCP3[i,3] < LGMinAFlow:
            
                                # if the habitat flow counter = 0
                                if LGAHCP3[i-1,5] == 0:
            
                                    # If the 3-day smolt flow trigger = 1 (trigger smolt flow)
                                    if LagunaSmoltTriggerCD[i] == 1:
                                        LGAHCP3[i,5] = 401
                                        LGAHCP3[i,6] = LGAMig[0,3]
            
                                    # If the 3-day smolt flow trigger = 0 (do not trigger smolt flow)   
                                    elif LagunaSmoltTriggerCD[i] == 0:
                                        LGAHCP3[i,6] = LGAHCP3[i,4]
            
                                # If the habitat flow counter = 360   
                                elif LGAHCP3[i-1,5] == 360:
            
                                    # If the 3-day smolt flow trigger = 1 (trigger smolt flow)
                                    if LagunaSmoltTriggerCD[i] == 1:
                                        LGAHCP3[i,5] = 401
                                        LGAHCP3[i,6] = LGAMig[0,3]
            
                                    # If the 3-day smolt flow trigger = 0 (do not trigger smolt flow)   
                                    elif LagunaSmoltTriggerCD[i] == 0:
                                        LGAHCP3[i,6] = LGAHCP3[i,4]
            
                                # If the habitat flow counter is >= 400       
                                elif LGAHCP3[i-1,5] >= 400:
            
                                    # If the 3-day smolt flow trigger = 1 (trigger smolt flow)
                                    if LagunaSmoltTriggerCD[i] == 1:
                                        LGAHCP3[i,5] = LGAHCP3[i-1,5] + 1
                                        LGAHCP3[i,6] = LGAMig[0,3]
            
                                    # If the 3-day smolt flow trigger = 0 (do not trigger smolt flow)   
                                    elif LagunaSmoltTriggerCD[i] == 0:
                                        LGAHCP3[i,6] = LGAHCP3[i,4]
            
                    # For the months of May we do not need to wory about
                    # flow values because adult migration does not occur
                    elif LGAHCP3[i,0] == 5:
            
                        # if the water year type is critically dry
                        if LGAHCP3[i,7] == 1:
            
                            # if the habitat flow counter = 0
                            if LGAHCP3[i-1,5] == 0:
            
                                # If the 3-day smolt flow trigger = 1 (trigger smolt flow)
                                if LagunaSmoltTriggerCD[i] == 1:
                                    LGAHCP3[i,5] = 401
                                    LGAHCP3[i,6] = LGAMig[0,3]
            
                                # If the 3-day smolt flow trigger = 0 (do not trigger smolt flow)   
                                elif LagunaSmoltTriggerCD[i] == 0:
                                    LGAHCP3[i,6] = LGAHCP3[i,4]
    
                            # If the habitat flow counter = 360
                            elif LGAHCP3[i-1,5] == 360:
    
                                # If the 3-day smolt flow trigger = 1 (trigger smolt flow)
                                if LagunaSmoltTriggerCD[i] == 1:
                                    LGAHCP3[i,5] = 401
                                    LGAHCP3[i,6] = LGAMig[0,3]
    
                                # If the 3-day smolt flow trigger = 0 (do not trigger smolt flow)
                                elif LagunaSmoltTriggerCD[i] == 0: 
                                    LGAHCP3[i,6] = LGAHCP3[i,4]
    
                            # If the habitat flow counter is >= 400
                            elif LGAHCP3[i-1,5] >= 400:
    
                                # If the 3-day smolt flow trigger = 1 (trigger smolt flow)
                                if LagunaSmoltTriggerCD[i] == 1:
                                    LGAHCP3[i,5] = LGAHCP3[i-1,5] + 1
                                    LGAHCP3[i,6] = LGAMig[0,3]
    
                                # If the 3-day smolt flow trigger = 0 (do not trigger smolt flow)
                                elif LagunaSmoltTriggerCD[i] == 0:
                                    LGAHCP3[i,6] = LGAHCP3[i,4]
    
            ## Set the 3-day smolt flow equal to the Tait array
            SmoltLDMJ = TaitSmoltTriggerCD
    
            ##### MAJORS CREEK
            # Section 3.3.2 Majors Non-Baseflow HCP Flow Rules
            # Same methods for Majors Creek
            # Majors HCP Tier I has no migration flows, so the flow goal is the 
            # baseflow goal. However, because the migration flows are based on the
            # flows unimpaired by the City's diversions, the outcome of following the
            # rules is based on the same flows for all three tiers. In order to handle
            # as much as possible in the fewest loops, I will run all three tiers for
            # Majors for Adult Migration in a single loop, with the "dur" duration flag
            # going into the Tier I array ONLY.
            # Function duration(date1, dur1, dur2, flow, goal)
            # dur1 and dur2 were the duration code of the prior day and two days, here 
            # they will be a column of flags retained for troubleshooting.
            MJAHCP3[:,6] = MJAHCP3[:,4]
    
            ## Majors HCP Flow Calculations
            for i in range(2, BTlength):
            
                # For the months December through April
                if MJAHCP3[i,0] == 12 or MJAHCP3[i,0] <= 4:
            
                    # ADULT MIGRATION
            
                    # For flows > the minimum adult migration flow
                    if MJAHCP3[i,3] >= MJMinAFlow:
            
                        # If the water year type >= 3
                        if MJAHCP3[i,7] >= 3:
            
                            # If the habitat flow counter equals zero
                            if MJAHCP3[i-1,5] == 0:
                                MJAHCP3[i,5] = 101
                                MJAHCP3[i,6] = MJAMig[0,0]
            
                            # If the habitat flow counter is >= 101 and < 200
                            elif MJAHCP3[i-1,5] >= 101 and MJAHCP3[i-1,5] < 200:
                                MJAHCP3[i,5] = MJAHCP3[i-1,5] + 1
                                MJAHCP3[i,6] = MJAMig[0,0]
            
                            # If the habitat flow counter is > 200
                            elif MJAHCP3[i-1,5] >= 200:
                                MJAHCP3[i,5] = 101
                                MJAHCP3[i,6] = MJAMig[0,0]
            
                        # Elseif the water year type <= 2
                        elif MJAHCP3[i,7] <= 2:
            
                            # Habitat flows = the minimum rearing flow of 0.25 cfs
                            MJAHCP3[i,6] = MJAMig[0,4]
            
                    # Gap in end related to space for comment above
            
                    # SPAWNING, INCUBATION AND SMOLT MIGRATION
            
                    # For flows < the minimum adult migration flow and the
                    elif MJAHCP3[i,3] < MJMinAFlow:
            
                        # If the water year type is >= 3
                        if MJAHCP3[i,7] >= 3:
            
                            # If the habitat flow counter equals zero
                            if MJAHCP3[i-1,5] == 0:
            
                                # Use the incubation flow for December
                                if MJAHCP3[i,0] == 12:
                                    
                                    MJAHCP3[i,6] = MJAMig[0,4]
            
                                # Use the smolt flow for January through March
                                else:
                                    
                                    MJAHCP3[i,6] = MJAMig[0,3]
            
                                # This If statement is to deal with times when
                                # hydrologic conditions change from dry, or worse,
                                # to average, or better conditions on the first of
                                # the month for the months January through April
            
                                # If the date is January 1st
                                if MJAHCP3[i,0] == 1 and MJAHCP3[i,1] == 1:
            
                                    # If cell i-1 WY type is dry or drier and cell i is average or wetter
                                    if MJAHCP3[i-1,7] <= 2 and MJAHCP3[i,7] >= 3:
            
                                        less = np.arange(13,0,-1)
                                        z = len(less)
                                        flow = MJAHCP3[i-less,3]
                                        flowgreater = [i for i in range(len(flow)) if flow[i] > MJMinAFlow]
                                        # positionmax = max(flowgreater)
                                        if np.size(flowgreater) > 0:
                                            positionmax = max(flowgreater)
                                        else:
                                            positionmax = []
                                        empty = not bool(positionmax)
            
                                        if not empty:
            
                                            MJAHCP3[i,5] = 201 + (z - (positionmax+1))
                                            MJAHCP3[i,6] = MJAMig[0,1]
            
                                elif MJAHCP3[i,0] == 2 and MJAHCP3[i,1] == 1:
            
                                    # If cell i-1 WY type is dry or drier and cell i is average or wetter
                                    if MJAHCP3[i-1,7] <= 2 and MJAHCP3[i,7] >= 3:
            
                                        less = np.arange(13,0,-1)
                                        z = len(less)
                                        flow = MJAHCP3[i-less,3]
                                        flowgreater = [i for i in range(len(flow)) if flow[i] > MJMinAFlow]
                                        # positionmax = max(flowgreater)
                                        # empty = bool(positionmax)
                                        if np.size(flowgreater) > 0:
                                            positionmax = max(flowgreater)
                                        else:
                                            positionmax = []
                                        empty = not bool(positionmax)
            
                                        if not empty:
            
                                            MJAHCP3[i,5] = 201 + (z - (positionmax+1))
                                            MJAHCP3[i,6] = MJAMig[0,1]
            
                                elif MJAHCP3[i,0] == 3 and MJAHCP3[i,1] == 1:
            
                                    # If cell i-1 WY type is dry or drier and cell i is average or wetter
                                    if MJAHCP3[i-1,7] <= 2 and MJAHCP3[i,7] >= 3:
            
                                        less = np.arange(13,0,-1)
                                        z = len(less)
                                        flow = MJAHCP3[i-less,3]
                                        flowgreater = [i for i in range(len(flow)) if flow[i] > MJMinAFlow]
                                        # positionmax = max(flowgreater)
                                        # empty = bool(positionmax)
                                        if np.size(flowgreater) > 0:
                                            positionmax = max(flowgreater)
                                        else:
                                            positionmax = []
                                        empty = not bool(positionmax)
            
                                        if not empty:
            
                                            MJAHCP3[i,5] = 201 + (z - (positionmax+1))
                                            MJAHCP3[i,6] = MJAMig[0,1]
            
                                elif MJAHCP3[i,0] == 4 and MJAHCP3[i,1] == 1: ##### SHAWN???
            
                                    # If cell i-1 WY type is dry or drier and cell i is average or wetter
                                    if MJAHCP3[i-1,7] <= 2 and MJAHCP3[i,7] >= 3:
            
                                        less = np.arange(13,0,-1)
                                        z = len(less)
                                        flow = MJAHCP3[i-less,3]
                                        flowgreater = [i for i in range(len(flow)) if flow[i] > 9]
                                        # positionmax = max(flowgreater)
                                        # empty = bool(positionmax)
                                        if np.size(flowgreater) > 0:
                                            positionmax = max(flowgreater)
                                        else:
                                            positionmax = []
                                        empty = not bool(positionmax)
            
                                        if not empty:
            
                                            MJAHCP3[i,5] = 201 + (z - (positionmax+1))
                                            MJAHCP3[i,6] = MJAMig[0,1]
            
                            # If the habitat flow counter is >= 101 and < 200
                            elif MJAHCP3[i-1,5] >= 101 and MJAHCP3[i-1,5] < 200:
                                MJAHCP3[i,5] = 201
                                MJAHCP3[i,6] = MJAMig[0,1]
            
                            # If the habitat flow counter is >= 201 and < 214
                            elif MJAHCP3[i-1,5] >= 201 and MJAHCP3[i-1,5] < 214:
                                MJAHCP3[i,5] = MJAHCP3[i-1,5] + 1
                                MJAHCP3[i,6] = MJAMig[0,1]
            
                            # If the habitat flow counter is == 214
                            elif MJAHCP3[i-1,5] == 214:
                                MJAHCP3[i,5] = 301
            
                                # Use the incubation flow for December
                                if MJAHCP3[i,0] == 12:
                                    MJAHCP3[i,6] = MJAMig[0,2]
            
                                # Use the smolt flow for January through March
                                else:
                                    MJAHCP3[i,6] = MJAMig[0,3]
            
                            # If the habitat flow counter is > 300 and < 360
                            elif MJAHCP3[i-1,5] > 300 and MJAHCP3[i-1,5] < 360:
                                MJAHCP3[i,5] = MJAHCP3[i-1,5] + 1
            
                                # Use the incubation flow for December
                                if MJAHCP3[i,0] == 12:
                                    MJAHCP3[i,6] = MJAMig[0,2]
            
                                # Use the smolt flow for January through March
                                else:
                                    MJAHCP3[i,6] = MJAMig[0,3]
            
                            # If the habitat flow counter = 360
                            elif MJAHCP3[i-1,5] == 360:
                                MJAHCP3[i,5] = 400
                                MJAHCP3[i,6] = MJAMig[0,3]
            
                            # If the habitat flow counter = 400
                            elif MJAHCP3[i-1,5] >= 400:
                                MJAHCP3[i,5] = 400
                                MJAHCP3[i,6] = MJAMig[0,3]
            
                        # If the water year type is <= 2
                        elif MJAHCP3[i,7] <= 2:
            
                            # Habitat flows = the minimum rearing flow of 0.25 cfs
                            MJAHCP3[i,6] = MJAMig[0,4]
            
                    # Gap in end related to space for comment above
            
                # Advancing to April adult migration flows stop and the rules for spawning and incubation are different,
                # also need to deal with times when flows exceed the adult migration flows
                elif MJAHCP3[i,0] == 5:
            
                    # SPAWNING, INCUBATION AND SMOLT MIGRATION
            
                    # For flows > the minimum adult migration flow
                    if MJAHCP3[i,3] >= MJMinAFlow:
            
                        # If the water year type is >= 3
                        if MJAHCP3[i,7] >= 3:
            
                            # If the habitat flow counter is >= 101 and < 200
                            if MJAHCP3[i-1,5] >= 101 and MJAHCP3[i-1,5] < 200:
                                MJAHCP3[i,5] = 201
                                MJAHCP3[i,6] = MJAMig[0,1]
            
                            # If the habitat flow counter is >= 201 and < 214
                            elif MJAHCP3[i-1,5] >= 201 and MJAHCP3[i-1,5] < 214:
                                MJAHCP3[i,5] = MJAHCP3[i-1,5] + 1
                                MJAHCP3[i,6] = MJAMig[0,1]
            
                            # If the habitat flow counter = 214
                            # Flow rule governed by smolt flow in this case
                            elif MJAHCP3[i-1,5] == 214:
                                MJAHCP3[i,5] = 301
                                MJAHCP3[i,6] = MJAMig[0,3]
            
                            # If the habitat flow counter is > 300 and < 360
                            # Flow rule governed by smolt flow in this case
                            elif MJAHCP3[i-1,5] > 300 and MJAHCP3[i-1,5] < 360:
                                MJAHCP3[i,5] = MJAHCP3[i-1,5] + 1
                                MJAHCP3[i,6] = MJAMig[0,3]
            
                            # If the habitat flow counter = 360
                            # Flow rule governed by smolt flow in this case
                            elif MJAHCP3[i-1,5] == 360:
                                MJAHCP3[i,5] = 400
                                MJAHCP3[i,6] = MJAMig[0,3]
            
                            # If the habitat flow counter = 400
                            # Flow rule governed by smolt flow in this case
                            elif MJAHCP3[i-1,5] >= 400:
                                MJAHCP3[i,5] = 400
                                MJAHCP3[i,6] = MJAMig[0,3]
            
                        # If the water year type is <= 2
                        elif MJAHCP3[i,7] <= 2:
            
                            # If the 3-day smolt flow trigger = 1 (trigger smolt flow)
                            if SmoltLDMJ[i] == 1:
            
                                # If the habitat flow counter = 0
                                if MJAHCP3[i-1,5] == 0:
                                    MJAHCP3[i,5] = 400
                                    MJAHCP3[i,6] = MJAMig[0,3]
            
                                # If the habitat flow counter >= 400
                                elif MJAHCP3[i-1,5] >= 400:
                                    MJAHCP3[i,5] = MJAHCP3[i-1,5] + 1
                                    MJAHCP3[i,6] = MJAMig[0,3]
            
                            # If the 3-day smolt flow trigger = 0 (do not trigger smolt flow)
                            elif SmoltLDMJ[i] == 0:
                                MJAHCP3[i,6] = MJAHCP3[i,4]
            
                    # For flows < the minimum adult migration flow
                    elif MJAHCP3[i,3] < MJMinAFlow:
            
                        # If the water year type is >= 3
                        if MJAHCP3[i,7] >= 3:
            
                            # If the habitat flow counter equals zero
                            if MJAHCP3[i-1,5] == 0:
                                MJAHCP3[i,6] = MJAMig[0,3]
            
                            # If the habitat flow counter is >= 101 and < 200
                            elif MJAHCP3[i-1,5] >= 101 and MJAHCP3[i-1,5] < 200:
                                MJAHCP3[i,5] = 201
                                MJAHCP3[i,6] = MJAMig[0,1]
            
                            # If the habitat flow counter is >= 201 and < 214
                            elif MJAHCP3[i-1,5] >= 201 and MJAHCP3[i-1,5] < 214:
                                MJAHCP3[i,5] = MJAHCP3[i-1,5] + 1
                                MJAHCP3[i,6] = MJAMig[0,1]
            
                            # If the habitat flow counter = 214
                            elif MJAHCP3[i-1,5] == 214:
                                MJAHCP3[i,5] = 301
                                MJAHCP3[i,6] = MJAMig[0,3]
            
                            # If the habitat flow counter is > 300 and < 360
                            elif MJAHCP3[i-1,5] > 300 and MJAHCP3[i-1,5] < 360:
                                MJAHCP3[i,5] = MJAHCP3[i-1,5] + 1
                                MJAHCP3[i,6] = MJAMig[0,3]
            
                            # If the habitat flow counter = 360
                            # Flow rule governed by smolt flow in this case
                            elif MJAHCP3[i-1,5] == 360:
                                MJAHCP3[i,5] = 400
                                MJAHCP3[i,6] = MJAMig[0,3]
            
                            # If the habitat flow counter = 400
                            # Flow rule governed by smolt flow in this case
                            elif MJAHCP3[i-1,5] == 400:
                                MJAHCP3[i,5] = 400
                                MJAHCP3[i,6] = MJAMig[0,3]
            
                            # This If statement is to deal with times when
                            # hydrologic conditions change from dry, or worse,
                            # to average, or better conditions on the first of
                            # the month for the months January through March
            
                            if MJAHCP3[i,0] == 5 and MJAHCP3[i,1] == 1:
            
                                # If cell i-1 WY type is dry or drier and cell i is average or wetter
                                if MJAHCP3[i-1,7] <= 2 and MJAHCP3[i,7] >= 3:
            
                                    less = np.arange(13,0,-1)
                                    z = len(less)
                                    flow = MJAHCP3[i-less,3]
                                    flowgreater = [i for i in range(len(flow)) if flow[i] > MJMinAFlow]
                                    # positionmax = max(flowgreater)
                                    # empty = bool(positionmax)
                                    if np.size(flowgreater) > 0:
                                        positionmax = max(flowgreater)
                                    else:
                                        positionmax = []
                                    empty = not bool(positionmax)
            
                                    if not empty:
            
                                        MJAHCP3[i,5] = 201 + (z - (positionmax+1))
                                        MJAHCP3[i,6] = MJAMig[0,1]
            
                        # If the water year type is <= 2
                        elif MJAHCP3[i,7] <= 2:
            
                            # If the 3-day smolt flow trigger = 1 (trigger smolt flow)
                            if SmoltLDMJ[i] == 1:
            
                                # If the habitat flow counter = 0
                                if MJAHCP3[i-1,5] == 0:
                                    MJAHCP3[i,5] = 400
                                    MJAHCP3[i,6] = MJAMig[0,3]
            
                                # If the habitat flow counter >= 400
                                elif MJAHCP3[i-1,5] >= 400:
                                    MJAHCP3[i,5] = MJAHCP3[i-1,5] + 1
                                    MJAHCP3[i,6] = MJAMig[0,3]
            
                            # If the 3-day smolt flow trigger = 0 (do not trigger smolt flow)
                            elif SmoltLDMJ[i] == 0:
                                MJAHCP3[i,6] = MJAHCP3[i,4]
            
                # For the months March through May to deal with the critically dry smolt rule
                if MJAHCP3[i,0] >= 3 and MJAHCP3[i,0] <= 5:
            
                    # SMOLT MIGRATION
            
                    # For the month of March or April
                    if MJAHCP3[i,0] == 3 or MJAHCP3[i,0] == 4:
            
                        # if the water year type is dry or critically dry
                        if MJAHCP3[i,7] <= 2:
            
                            # if the habitat flow counter = 0
                            if MJAHCP3[i-1,5] <= 360:
            
                                # If the 3-day smolt flow trigger = 1 (trigger smolt flow)
                                if SmoltLDMJ[i] == 1:
                                    MJAHCP3[i,5] = 401
                                    MJAHCP3[i,6] = MJAMig[0,3]
            
                                # If the 3-day smolt flow trigger = 0 (do not trigger smolt flow)
                                elif SmoltLDMJ[i] == 0:
                                    MJAHCP3[i,6] = MJAHCP3[i,4]
            
                            # If the habitat flow counter is >= 400
                            elif MJAHCP3[i-1,5] >= 400:
            
                                # If the 3-day smolt flow trigger = 1 (trigger smolt flow)
                                if SmoltLDMJ[i] == 1:
                                    MJAHCP3[i,5] = MJAHCP3[i-1,5] + 1
                                    MJAHCP3[i,6] = MJAMig[0,3]
            
                                # If the 3-day smolt flow trigger = 0 (do not trigger smolt flow)
                                elif SmoltLDMJ[i] == 0:
                                    MJAHCP3[i,6] = MJAHCP3[i,4]
            
                    # For the month of May we do not need to wory about
                    # flow values because adult migration does not occur
                    elif MJAHCP3[i,0] == 5:
            
                        # if the water year type is dry or critically dry
                        if MJAHCP3[i,7] <= 2:
            
                            # if the habitat flow counter = 0
                            if MJAHCP3[i-1,5] <= 360:
            
                                # If the 3-day smolt flow trigger = 1 (trigger smolt flow)
                                if SmoltLDMJ[i] == 1:
                                    MJAHCP3[i,5] = 401
                                    MJAHCP3[i,6] = MJAMig[0,3]
    
                                # If the 3-day smolt flow trigger = 0 (do not trigger smolt flow)
                                elif SmoltLDMJ[i] == 0:
                                    MJAHCP3[i,6] = MJAHCP3[i,4]
    
    
                            # If the habitat flow counter is >= 400
                            if MJAHCP3[i-1,6] >= 400:
    
                                # If the 3-day smolt flow trigger = 1 (trigger smolt flow)
                                if SmoltLDMJ[i] == 1:
                                    MJAHCP3[i,5] = MJAHCP3[i-1,6] + 1
                                    MJAHCP3[i,6] = MJAMig[0,3]
    
                                # If the 3-day smolt flow trigger = 0 (do not trigger smolt flow)
                                elif SmoltLDMJ[i] == 0:
                                    MJAHCP3[i,6] = MJAHCP3[i,4]
    
            ##### LIDDELL CREEK 
            # Section 3.3.3 Liddell Non-Basefloe HCP Flow Rules
            # Liddell HCP Tier I has no migration flows, so the flow goal is the 
            # baseflow goal. However, because the migration flows are based on the
            # flows unimpaired by the City's diversions, the outcome of following the
            # rules is based on the same flows for all three tiers. In order to handle
            # as much as possible in the fewest loops, I will run all three tiers for
            # Liddell for Adult Migration in a single loop, with the "dur" duration 
            # flag going into the Tier I array ONLY.
            
            # Function duration(date1, dur1, dur2, flow, goal)
            # dur1 and dur2 were the duration code of the prior day and two days, 
            # here they will be a column of flags retained for troubleshooting.
            LDAHCP3[:,6] = LDAHCP3[:,4] 
            
            # ' Setting the Adult Migration Trigger
            # I've copied the code from Majors for Liddell, so the extra work
            # here is not needed at this time, but is in place in case JH
            # changes the rules again.
            # I'll work on Tier II first.
            # ' Setting the Adult Migration Trigger
            # Liddell Smolt outmigration bypass is equal to the incubation flow, but
            # smaller than the highest baseflow bypass. Smolt is only used in Tier III
            # years, and because of these complications, I have to handle the tiers
            # with separate IF blocks.
            # I'll work on Tier II first. Adult migration in Dec for Tier II during
            # three wettest year types.
    
            ## LIDDELL HCP CALCULATIONS
            for i in range(2, BTlength):
            
                # For the months December through April
                if LDAHCP3[i,0] == 12 or LDAHCP3[i,0] <= 4:
            
                    # ADULT MIGRATION
            
                    # For flows > the minimum adult migration flow
                    if LDAHCP3[i,3] >= LDMinAFlow:
            
                        # If the water year type >= 3
                        if LDAHCP3[i,7] >= 3:
            
                            # If the habitat flow counter equals zero
                            if LDAHCP3[i-1,5] == 0:
                                LDAHCP3[i,5] = 101
                                LDAHCP3[i,6] = LDAMig[0,0]
            
                            # If the habitat flow counter is >= 101 and < 200
                            elif LDAHCP3[i-1,5] >= 101 and LDAHCP3[i-1,5] < 200:
                                LDAHCP3[i,5] = LDAHCP3[i-1,5] + 1
                                LDAHCP3[i,6] = LDAMig[0,0]
            
                            # If the habitat flow counter is >= 200
                            elif LDAHCP3[i-1,5] >= 200:
                                LDAHCP3[i,5] = 101
                                LDAHCP3[i,6] = LDAMig[0,0]
            
                        # Elseif the water year type <= 2
                        elif LDAHCP3[i,7] <= 2:
            
                            # Habitat flows = the minimum rearing flow of 0.25 cfs
                            LDAHCP3[i,6] = LDAMig[0,4]
            
                    # SPAWNING, INCUBATION AND SMOLT MIGRATION
            
                    # For flows < the minimum adult migration flow and the
                    elif LDAHCP3[i,3] < LDMinAFlow:
            
                        # If the water year type is >= 3
                        if LDAHCP3[i,7] >= 3:
            
                            # If the habitat flow counter equals zero
                            if LDAHCP3[i-1,5] == 0:
            
                                # Use the incubation flow for December
                                if LDAHCP3[i,0] == 12:
                                    LDAHCP3[i,6] = LDAMig[0,4]
            
                                # Use the smolt flow for January through March
                                else:
                                    LDAHCP3[i,6] = LDAMig[0,3]
            
                                # This If statement is to deal with times when
                                # hydrologic conditions change from dry, or worse,
                                # to average, or better conditions on the first of
                                # the month for the months January through March
            
                                # If the date is January 1st
                                if LDAHCP3[i,0] == 1 and LDAHCP3[i,1] == 1:
            
                                    # If cell i-1 WY type is dry or drier and cell i is average or wetter
                                    if LDAHCP3[i-1,7] <= 2 and LDAHCP3[i,7] >= 3:
            
                                        less = np.arange(13,0,-1)
                                        z = len(less)
                                        flow = LDAHCP3[i-less,3]
                                        flowgreater = [i for i in range(len(flow)) if flow[i] > LDMinAFlow]
                                        # positionmax = max(flowgreater)
                                        # empty = bool(positionmax)
                                        if np.size(flowgreater) > 0:
                                            positionmax = max(flowgreater)
                                        else:
                                            positionmax = []
                                        empty = not bool(positionmax)
            
                                        if not empty:
            
                                            LDAHCP3[i,5] = 201 + (z - (positionmax+1))
                                            LDAHCP3[i,6] = LDAMig[0,1]
            
                                elif LDAHCP3[i,0] == 2 and LDAHCP3[i,1] == 1:
            
                                    # If cell i-1 WY type is dry or drier and cell i is average or wetter
                                    if LDAHCP3[i-1,7] <= 2 and LDAHCP3[i,7] >= 3:
            
                                        less = np.arange(13,0,-1)
                                        z = len(less)
                                        flow = LDAHCP3[i-less,3]
                                        flowgreater = [i for i in range(len(flow)) if flow[i] > LDMinAFlow]
                                        # positionmax = max(flowgreater)
                                        # empty = bool(positionmax)
                                        if np.size(flowgreater) > 0:
                                            positionmax = max(flowgreater)
                                        else:
                                            positionmax = []
                                        empty = not bool(positionmax)
            
                                        if not empty:
            
                                            LDAHCP3[i,5] = 201 + (z - (positionmax+1))
                                            LDAHCP3[i,6] = LDAMig[0,1]
            
                                elif LDAHCP3[i,0] == 3 and LDAHCP3[i,1] == 1:
            
                                    # If cell i-1 WY type is dry or drier and cell i is average or wetter
                                    if LDAHCP3[i-1,7] <= 2 and LDAHCP3[i,7] >= 3:
            
                                        less = np.arange(13,0,-1)
                                        z = len(less)
                                        flow = LDAHCP3[i-less,3]
                                        flowgreater = [i for i in range(len(flow)) if flow[i] > LDMinAFlow]
                                        # positionmax = max(flowgreater)
                                        # empty = bool(positionmax)
                                        if np.size(flowgreater) > 0:
                                            positionmax = max(flowgreater)
                                        else:
                                            positionmax = []
                                        empty = not bool(positionmax)
            
                                        if not empty:
            
                                            LDAHCP3[i,5] = 201 + (z - (positionmax+1))
                                            LDAHCP3[i,6] = LDAMig[0,1]
            
                                elif LDAHCP3[i,0] == 4 and LDAHCP3[i,1] == 1: ##### SHAWN???
            
                                    # If cell i-1 WY type is dry or drier and cell i is average or wetter
                                    if LDAHCP3[i-1,7] <= 2 and LDAHCP3[i,7] >= 3:
            
                                        less = np.arange(13,0,-1)
                                        z = len(less)
                                        flow = LDAHCP3[i-less,3]
                                        flowgreater = [i for i in range(len(flow)) if flow[i] > LDMinAFlow]
                                        # positionmax = max(flowgreater)
                                        # empty = bool(positionmax)
                                        if np.size(flowgreater) > 0:
                                            positionmax = max(flowgreater)
                                        else:
                                            positionmax = []
                                        empty = not bool(positionmax)
            
                                        if not empty:
            
                                            LDAHCP3[i,5] = 201 + (z - (positionmax+1))
                                            LDAHCP3[i,6] = LDAMig[0,1]
            
                            # If the habitat flow counter is >= 101 and < 200
                            elif LDAHCP3[i-1,5] >= 101 and LDAHCP3[i-1,5] < 200:
                                LDAHCP3[i,5] = 201
                                LDAHCP3[i,6] = LDAMig[0,1]
            
                            # If the habitat flow counter is >= 201 and < 214  ##### SHAWN??? SHOULD THIS BE >=200 or 201???
                            elif LDAHCP3[i-1,5] >= 201 and LDAHCP3[i-1,5] < 214:
                                LDAHCP3[i,5] = LDAHCP3[i-1,5] + 1
                                LDAHCP3[i,6] = LDAMig[0,1]
            
                            # If the habitat flow counter is == 214
                            elif LDAHCP3[i-1,5] == 214:
                                LDAHCP3[i,5] = 301
            
                                # Use the incubation flow for December
                                if LDAHCP3[i,0] == 12:
                                    LDAHCP3[i,6] = LDAMig[0,2]
            
                                # Use the smolt flow for January through March
                                else:
                                    LDAHCP3[i,6] = LDAMig[0,3]
            
                            # If the habitat flow counter is > 300 and < 360
                            elif LDAHCP3[i-1,5] > 300 and LDAHCP3[i-1,5] < 360:
                                LDAHCP3[i,5] = LDAHCP3[i-1,5] + 1
            
                                # Use the incubation flow for December
                                if LDAHCP3[i,0] == 12:
                                    LDAHCP3[i,6] = LDAMig[0,2]
            
                                # Use the smolt flow for January through March
                                else:
                                    LDAHCP3[i,6] = LDAMig[0,3]
            
                            # If the habitat flow counter = 360
                            elif LDAHCP3[i-1,5] == 360:
                                LDAHCP3[i,5] = 400
                                LDAHCP3[i,6] = LDAMig[0,3]
            
                            # If the habitat flow counter = 400
                            elif LDAHCP3[i-1,5] >= 400:
                                LDAHCP3[i,5] = 400
                                LDAHCP3[i,6] = LDAMig[0,3]
            
                        # If the water year type is <= 2
                        elif LDAHCP3[i,7] <= 2:
            
                            # Habitat flows = the minimum rearing flow of 0.25 cfs
                            LDAHCP3[i,6] = LDAMig[0,4]
            
                # Advancing to April adult migration flows stop and the rules for spawning and incubation are different,
                # also need to deal with times when flows exceed the adult migration flows
                elif LDAHCP3[i,0] == 5:
            
                    # SPAWNING, INCUBATION AND SMOLT MIGRATION
            
                    # For flows > the minimum adult migration flow
                    if LDAHCP3[i,3] >= LDMinAFlow:
            
                        # If the water year type is >= 3
                        if LDAHCP3[i,7] >= 3:
            
                            # If the habitat flow counter is >= 101 and < 200
                            if LDAHCP3[i-1,5] >= 101 and LDAHCP3[i-1,5] < 200:
                                LDAHCP3[i,5] = 201
                                LDAHCP3[i,6] = LDAMig[0,1]
            
                            # If the habitat flow counter is >= 200 and < 214
                            elif LDAHCP3[i-1,5] >= 201 and LDAHCP3[i-1,5] < 214:
                                LDAHCP3[i,5] = LDAHCP3[i-1,5] + 1
                                LDAHCP3[i,6] = LDAMig[0,1]
            
                            # If the habitat flow counter = 214
                            # Flow rule governed by smolt flow in this case
                            elif LDAHCP3[i-1,5] == 214:
                                LDAHCP3[i,5] = 301
                                LDAHCP3[i,6] = LDAMig[0,3]
            
                            # If the habitat flow counter is > 300 and < 360
                            # Flow rule governed by smolt flow in this case
                            elif LDAHCP3[i-1,5] > 300 and LDAHCP3[i-1,5] < 360:
                                LDAHCP3[i,5] = LDAHCP3[i-1,5] + 1
                                LDAHCP3[i,6] = LDAMig[0,3]
            
                            # This If statement is to deal with times when
                            # hydrologic conditions change from dry, or worse,
                            # to average, or better conditions on the first of
                            # the month for the months January through March
            
                            if LDAHCP3[i,0] == 4 and LDAHCP3[i,1] == 1:
            
                                # If cell i-1 WY type is dry or drier and cell i is average or wetter
                                if LDAHCP3[i-1,7] <= 2 and LDAHCP3[i,7] >= 3:
            
                                    less = np.arange(13,0,-1)
                                    z = len(less)
                                    flow = LDAHCP3[i-less,3]
                                    flowgreater = [i for i in range(len(flow)) if flow[i] > LDMinAFlow]
                                    # positionmax = max(flowgreater)
                                    # empty = bool(positionmax)
                                    if np.size(flowgreater) > 0:
                                        positionmax = max(flowgreater)
                                    else:
                                        positionmax = []
                                    empty = not bool(positionmax)
            
                                    if not empty:
            
                                        LDAHCP3[i,5] = 201 + (z - (positionmax+1))
                                        LDAHCP3[i,6] = LDAMig[0,1]
            
                        # If the water year type is <= 2
                        elif LDAHCP3[i,7] <= 2:
            
                            # If the 3-day smolt flow trigger = 1 (trigger smolt flow)
                            if SmoltLDMJ[i] == 1:
            
                                # If the habitat flow counter = 0
                                if LDAHCP3[i-1,5] == 0:
                                    LDAHCP3[i,5] = 400
                                    LDAHCP3[i,6] = LDAMig[0,3]
            
                                # If the habitat flow counter >= 400
                                elif LDAHCP3[i-1,5] >= 400:
                                    LDAHCP3[i,5] = LDAHCP3[i-1,5] + 1
                                    LDAHCP3[i,6] = LDAMig[0,3]
            
                            # If the 3-day smolt flow trigger = 0 (do not trigger smolt flow)
                            elif SmoltLDMJ[i] == 0:
                                LDAHCP3[i,6] = LDAHCP3[i,4]
            
                    # For flows < the minimum adult migration flow
                    elif LDAHCP3[i,3] < LDMinAFlow:
            
                        # If the water year type is >= 3
                        if LDAHCP3[i,7] >= 3:
            
                            # If the habitat flow counter equals zero
                            if LDAHCP3[i-1,5] == 0:
                                LDAHCP3[i,6] = LDAMig[0,3]
            
                            # If the habitat flow counter is >= 101 and < 200
                            elif LDAHCP3[i-1,5] >= 101 and LDAHCP3[i-1,5] < 200:
                                LDAHCP3[i,5] = 201
                                LDAHCP3[i,6] = LDAMig[0,1]
            
                            # If the habitat flow counter is >= 201 and < 214
                            elif LDAHCP3[i-1,5] >= 201 and LDAHCP3[i-1,5] < 214:
                                LDAHCP3[i,5] = LDAHCP3[i-1,5] + 1
                                LDAHCP3[i,6] = LDAMig[0,1]
            
                            # If the habitat flow counter = 214
                            elif LDAHCP3[i-1,5] == 214:
                                LDAHCP3[i,5] = 301
                                LDAHCP3[i,6] = LDAMig[0,3]
            
                            # If the habitat flow counter is > 300 and < 360
                            elif LDAHCP3[i-1,5] > 300 and LDAHCP3[i-1,5] < 360:
                                LDAHCP3[i,5] = LDAHCP3[i-1,5] + 1
                                LDAHCP3[i,6] = LDAMig[0,3]
    
                            # This If statement is to deal with times when
                            # hydrologic conditions change from dry, or worse,
                            # to average, or better conditions on the first of
                            # the month for the months January through March
    
                            if LDAHCP3[i,0] == 4 and LDAHCP3[i,1] == 1:
    
                                # If cell i-1 WY type is dry or drier and cell i is average or wetter
                                if LDAHCP3[i-1,7] <= 2 and LDAHCP3[i,7] >= 3:
            
                                    less = np.arange(13,0,-1)
                                    z = len(less)
                                    flow = LDAHCP3[i-less,3]
                                    flowgreater = [i for i in range(len(flow)) if flow[i] > LDMinAFlow]
                                    positionmax = max(flowgreater)
                                    empty = bool(positionmax)
            
                                    if not empty:
            
                                        LDAHCP3[i,5] = 201 + (z - (positionmax+1))
                                        LDAHCP3[i,6] = LDAMig[0,1]
                            
                        # If the water year type is <= 2
                        elif LDAHCP3[i,7] <= 2:
    
                            # If the 3-day smolt flow trigger = 1 (trigger smolt flow)
                            if SmoltLDMJ[i] == 1:
    
                                # If the habitat flow counter = 0
                                if LDAHCP3[i-1,5] == 0:
                                    LDAHCP3[i,5] = 400
                                    LDAHCP3[i,6] = LDAMig[0,3]
    
                                # If the habitat flow counter >= 400#
                                elif LDAHCP3[i-1,5] >= 400:
                                    LDAHCP3[i,5] = LDAHCP3[i-1,5] + 1
                                    LDAHCP3[i,6] = LDAMig[0,3]
    
                            # If the 3-day smolt flow trigger = 0 (do not trigger smolt flow)#
                            elif SmoltLDMJ[i] == 0:
                                LDAHCP3[i,6] = LDAHCP3[i,4]
                   
                # For the months March through May to deal with the critically dry smolt rule
                if LDAHCP3[i,0] >= 3 and LDAHCP3[i,0] <= 5:
            
                    # SMOLT MIGRATION
            
                    # For the month of March or April
                    if LDAHCP3[i,0] == 3 or LDAHCP3[i,0] == 4:
            
                        # if the water year type is dry or critically dry
                        if LDAHCP3[i,7] <= 2:
            
                            # if the habitat flow counter = 0
                            if LDAHCP3[i-1,5] <= 360:
            
                                # If the 3-day smolt flow trigger = 1 (trigger smolt flow)
                                if SmoltLDMJ[i] == 1:
                                    LDAHCP3[i,5] = 401
                                    LDAHCP3[i,6] = LDAMig[0,3]
            
                                # If the 3-day smolt flow trigger = 0 (do not trigger smolt flow)
                                elif SmoltLDMJ[i] == 0:
                                    LDAHCP3[i,6] = LDAHCP3[i,4]
            
                            # If the habitat flow counter is >= 400
                            elif LDAHCP3[i-1,5] >= 400:
            
                                # If the 3-day smolt flow trigger = 1 (trigger smolt flow)
                                if SmoltLDMJ[i] == 1:
                                    LDAHCP3[i,5] = LDAHCP3[i-1,5] + 1
                                    LDAHCP3[i,6] = LDAMig[0,3]
            
                                # If the 3-day smolt flow trigger = 0 (do not trigger smolt flow)
                                elif SmoltLDMJ[i] == 0:
                                    LDAHCP3[i,6] = LDAHCP3[i,4]
            
                    # For the months of April and May we do not need to wory about
                    # flow values because adult migration does not occur
                    elif LDAHCP3[i,0] == 5:
            
                        # if the water year type is dry or critically dry
                        if LDAHCP3[i,7] <= 2:
            
                            # if the habitat flow counter = 0
                            if LDAHCP3[i-1,5] <= 360:
            
                                # If the 3-day smolt flow trigger = 1 (trigger smolt flow)
                                if SmoltLDMJ[i] == 1:
                                    LDAHCP3[i,5] = 401
                                    LDAHCP3[i,6] = LDAMig[0,3]
            
                                # If the 3-day smolt flow trigger = 0 (do not trigger smolt flow)
                                elif SmoltLDMJ[i] == 0:
                                    LDAHCP3[i,6] = LDAHCP3[i,4]
                                    
                            # If the habitat flow counter is >= 400
                            elif LDAHCP3[i-1,5] >= 400:
    
                                # If the 3-day smolt flow trigger = 1 (trigger smolt flow)
                                if SmoltLDMJ[i] == 1:
                                    LDAHCP3[i,5] = LDAHCP3[i-1,5] + 1
                                    LDAHCP3[i,6] = LDAMig[0,3]
    
                                # If the 3-day smolt flow trigger = 0 (do not trigger smolt flow)
                                elif SmoltLDMJ[i] == 0:
                                    LDAHCP3[i,6] = LDAHCP3[i,4]
    
            #####  Section 3.4 Use Baseflow Rules when Baseflow Rules require more bypass
            # Now run through all the flows and choose Baseflow to replace
            # Migration/Spawning/Incubate/Smolt flows if Baseflow is greater than those
            # flows.
            
            for i in range(BTlength):
            
                if TaitHCP3[i,3] > TaitHCP3[i,4]:
                    TaitHCP3[i,4] = TaitHCP3[i,3]
            
                if LGAHCP3[i,4] > LGAHCP3[i,6]:
                    LGAHCP3[i,6] = LGAHCP3[i,4]
            
                if MJAHCP3[i,4] > MJAHCP3[i,6]:
                    MJAHCP3[i,6] = MJAHCP3[i,4]
            
                if LDAHCP3[i,4] > LDAHCP3[i,6]:
                    LDAHCP3[i,6] = LDAHCP3[i,4]
            
            # At this point I have the anadromous flow goals for Tait, Laguna, Majors,
            # and Liddell. Newell will be delt with later, at this point we have no
            # flow goals for that reach. The final step is to convert the flows in the
            # anadromous reaches to the find the available flows at the points of
            # diversion.                      
            
            ##### Section 3.6 Upper Stations under HCP Flow Rules
            # To start with I'm using a simple 1:1 relationship of flows needed at the
            # anadromous stations to available at the upper stations. The logic is
            # based on observations of maintenance releases at the diversions resulting
            # in approximatly equivalent increases in flow in the anadromous stations.
            
            # Create preallocated arrays for all parameters for increased code efficiency
            
            # BTHCP, Create arrays for the Upstream stations, available flows under the
            # HCP rules, a different one for each of the three Tiers.
            BTHCP3 = np.zeros((BTlength,4))
            BTHCP3[:,:3] = BT[:,:3]
            
            # model_V8 addtion --
            # On the SLR we have two points of diversion, so we need to keep track of
            # what flow isn't available at the u/s station, but is available at the
            # downstream station.
            # Column 4 = additional flow below BigTrees due to HCP
            # Column 5 = adnl flow below BT due to the pre-existing leagal bypass (PX)
            # Column 6 = total additional flow below BigTrees (sum of HCP and PX)
            TBTait3 = np.zeros((BTlength,6))
            TBTait3[:,:3] = BT[:,:3]
            TBTait3[:,3:6] = 0
            # end of model_V8 addition --
                    
            # Initialize the arrays LGHCP3, MJHCP3, and LDHCP3 with the same shape as LGc, MJc, and LDc
            LGHCP3 = np.zeros_like(LGc)
            MJHCP3 = np.zeros_like(MJc)
            LDHCP3 = np.zeros_like(LDc)
            
            # Copy the first three columns from LGc to LGHCP3
            LGHCP3[:, :3] = LGc[:, :3]
            
            # Set the fourth column of LGHCP3 to zeros
            LGHCP3[:, 3] = np.zeros(BTlength)
            
            # Copy the first three columns from MJc to MJHCP3
            MJHCP3[:, :3] = MJc[:, :3]
            
            # Set the fourth column of MJHCP3 to zeros
            MJHCP3[:, 3] = np.zeros(BTlength)
            
            # Copy the first three columns from LDc to LDHCP3
            LDHCP3[:, :3] = LDc[:, :3]
            
            # Set the fourth column of LDHCP3 to zeros
            LDHCP3[:, 3] = np.zeros(BTlength)                            
    
            # Calculate the available flow at the upstream station.
            # 5/27/11 Version 10 corrects a long running error of not using the Big
            # Trees record that was corrected for the Felton Diversions. This should
            # have the largest impact on the "TBTait" array, as it effects the
            # calculation of additional flow available for diversion below the Big
            # Trees diversion.
            # Felton Changes take affect here. First, pre-allocate to new Felton 
            # data array.
            Felton = np.zeros((BTlength,9))
            Felton[:,:3] = BT[:,:3]
            # Column 4 is the counter, column 5 is the flow rule
            Felton[:,3:6] = np.zeros((BTlength,3))
            Felton[:,6] = BTUn[:,3]
            Felton[:,7] = TaitHCP3[:,7]
            Felton[:,8] = np.zeros((BTlength))
            # Enter the loop as before
            for i in range(BTlength):
                if i == 0:
                    TaitHCP3[i,6] = 0.01
                # FELTON MOD MADE HERE
                # Felton rule takes affect here and modifies the flow available at BT
                # according to the adult migration requirement of 40 cfs at Felton.
                # Felton also adds a 14-day spawning period following each migration 
                # event. Felton diversion located a few hundred meters upstream of the 
                # Big Trees gage. In all case the new rules must preserve existing 
                # diversion rules and operations at Tait Street.
            
                # Enter if statements. First statement asks if flow available for
                # diversion at Tait Street is > 0. Keep this same logic because Felton
                # rules will not take effect unless Tait is active. The logical Or
                # statement added with the May 2017 modifications to account for
                # removing the CDFW 5% rule.
                if TaitHCP3[i,6] > 0 or Felton[i-1,3] >= 101:
            
                    # If flow at Big Trees is > than FeltonMig of 40 cfs
                    if BTUn[i,3] >= FeltonMig:
            
                        # For the months December through April
                        if TaitHCP3[i,0] == 12 or TaitHCP3[i,0] <= 4:
            
                            if Felton[i-1,3] == 0 or Felton[i-1,3] == 99:
                                Felton[i,3] = 100
                                Felton[i,4] = FeltonMig
                                # BT available is BT unimpaired less Felton bypass
                                BTHCP3[i,3] = BTUn[i,3] - FeltonMig
                                Felton[i,5] = BTHCP3[i,3]
                                Felton[i,8] = TaitHCP3[i,6]
                                # When bypassing 40 cfs, available at Tait equals total 
                                # available at Tait (per Tait flow rules) less available
                                # at BT, as just computed 
                                TBTait3[i,3] = TaitHCP3[i,6] - BTHCP3[i,3]
            
                            elif Felton[i-1,3] >= 100 and Felton[i-1,3] < 114:
                                Felton[i,3] = 100
                                Felton[i,4] = FeltonMig
                                # BT available is BT unimpaired less Felton bypass
                                BTHCP3[i,3] = BTUn[i,3] - FeltonMig
                                Felton[i,5] = BTHCP3[i,3]
                                Felton[i,8] = TaitHCP3[i,6]
                                # When bypassing 40 cfs, available at Tait equals total 
                                # available at Tait (per Tait flow rules) less available
                                # at BT, as just computed
                                TBTait3[i,3] = TaitHCP3[i,6] - BTHCP3[i,3]
            
                            elif Felton[i-1,3] == 114:
                                Felton[i,3] = 100
                                Felton[i,4] = FeltonMig
                                # BT available is BT unimpaired less Felton bypass
                                BTHCP3[i,3] = BTUn[i,3] - FeltonMig
                                Felton[i,5] = BTHCP3[i,3]
                                Felton[i,8] = TaitHCP3[i,6]
                                # When bypassing 40 cfs, available at Tait equals total 
                                # available at Tait (per Tait flow rules) less available
                                # at BT, as just computed
                                TBTait3[i,3] = TaitHCP3[i,6] - BTHCP3[i,3]
            
                        elif TaitHCP3[i,0] == 5:
            
                            if Felton[i-1,3] >= 100 and Felton[i-1,3] < 114:
                                Felton[i,3] = Felton[i-1,3] + 1
                                Felton[i,4] = FeltonMig
                                # BT available is BT unimpaired less Felton bypass
                                BTHCP3[i,3] = BTUn[i,3] - FeltonMig
                                Felton[i,5] = BTHCP3[i,3]
                                Felton[i,8] = TaitHCP3[i,6]
                                # When bypassing 40 cfs, available at Tait equals total 
                                # available at Tait (per Tait flow rules) less available
                                # at BT, as just computed
                                TBTait3[i,3] = TaitHCP3[i,6] - BTHCP3[i,3]
            
                            elif Felton[i-1,3] == 114:
                                Felton[i,3] = 99
                                Felton[i,4] = 0
                                # BT available is BT unimpaired less Felton bypass
                                BTHCP3[i,3] = TaitHCP3[i,6]
                                Felton[i,5] = BTHCP3[i,3]
                                Felton[i,8] = TaitHCP3[i,6]
                                # When bypassing 40 cfs, available at Tait equals total 
                                # available at Tait (per Tait flow rules) less available
                                # at BT, as just computed
                                TBTait3[i,3] = TaitHCP3[i,6] - BTHCP3[i,3]
            
                            elif Felton[i-1,3] == 99:
                                Felton[i,3] = 99
                                Felton[i,4] = 0
                                # BT available is BT unimpaired less Felton bypass
                                BTHCP3[i,3] = TaitHCP3[i,6]
                                Felton[i,5] = BTHCP3[i,3]
                                Felton[i,8] = TaitHCP3[i,6]
                                # When bypassing 40 cfs, available at Tait equals total 
                                # available at Tait (per Tait flow rules) less available
                                # at BT, as just computed
                                TBTait3[i,3] = TaitHCP3[i,6] - BTHCP3[i,3]
            
                        else:
            
                            # This block of code provides the previously applied logic
                            # to compute available flow at Big Trees
                            if BTUn[i,3] > TaitHCP3[i,6]:
            
                                BTHCP3[i,3] = TaitHCP3[i,6]
                                Felton[i,5] = BTHCP3[i,3]
                                Felton[i,8] = TaitHCP3[i,6]
            
                            else:
            
                                BTHCP3[i,3] = BTUn[i,3]
                                TBTait3[i,3] = TaitHCP3[i,6] - BTUn[i,3]
                                Felton[i,5] = BTHCP3[i,3]
                                Felton[i,8] = TaitHCP3[i,6] - BTUn[i,3]
            
                    # If flow at BT < FeltonMig of 40 cfs
                    elif BTUn[i,3] < FeltonMig:
            
                        # For the months December through April
                        if TaitHCP3[i,0] == 12 or TaitHCP3[i,0] <= 4:
            
                            if Felton[i-1,3] >= 100 and Felton[i-1,3] < 114:
                                Felton[i,3] = Felton[i-1,3] + 1
                                Felton[i,4] = FeltonMig
                                # BT available is BT unimpaired less Felton bypass
                                BTHCP3[i,3] = 0
                                Felton[i,5] = BTHCP3[i,3]
                                Felton[i,8] = TaitHCP3[i,6]
                                # When bypassing 40 cfs, available at Tait equals total 
                                # available at Tait (per Tait flow rules) less available
                                # at BT, as just computed
                                TBTait3[i,3] = TaitHCP3[i,6] - BTHCP3[i,3]
            
                            elif Felton[i-1,3] == 114:
                                Felton[i,3] = 99
                                Felton[i,4] = 0
                                # BT available is BT unimpaired less Felton bypass
                                BTHCP3[i,3] = TaitHCP3[i,6]
                                Felton[i,5] = BTHCP3[i,3]
                                Felton[i,8] = TaitHCP3[i,6]
                                # When bypassing 40 cfs, available at Tait equals total 
                                # available at Tait (per Tait flow rules) less available
                                # at BT, as just computed
                                TBTait3[i,3] = TaitHCP3[i,6] - BTHCP3[i,3]
            
                            elif Felton[i-1,3] == 99:
                                Felton[i,3] = 99
                                Felton[i,4] = 0
                                # BT available is BT unimpaired less Felton bypass
                                BTHCP3[i,3] = TaitHCP3[i,6]
                                Felton[i,5] = BTHCP3[i,3]
                                Felton[i,8] = TaitHCP3[i,6]
                                # When bypassing 40 cfs, available at Tait equals total 
                                # available at Tait (per Tait flow rules) less available
                                # at BT, as just computed
                                TBTait3[i,3] = TaitHCP3[i,6] - BTHCP3[i,3]
            
                            elif Felton[i-1,3] == 0:
                                Felton[i,3] = 99
                                Felton[i,4] = 0
                                # BT available is BT unimpaired less Felton bypass
                                BTHCP3[i,3] = TaitHCP3[i,6]
                                Felton[i,5] = BTHCP3[i,3]
                                Felton[i,8] = TaitHCP3[i,6]
                                # When bypassing 40 cfs, available at Tait equals total 
                                # available at Tait (per Tait flow rules) less available
                                # at BT, as just computed 
                                TBTait3[i,3] = TaitHCP3[i,6] - BTHCP3[i,3]
            
                        elif TaitHCP3[i,0] == 5:
            
                            if Felton[i-1,3] >= 100 and Felton[i-1,3] < 114:
                                Felton[i,3] = Felton[i-1,3] + 1
                                Felton[i,4] = FeltonMig
                                # BT available is BT unimpaired less Felton bypass
                                BTHCP3[i,3] = 0
                                Felton[i,5] = BTHCP3[i,3]
                                Felton[i,8] = TaitHCP3[i,6]
                                # When bypassing 40 cfs, available at Tait equals total 
                                # available at Tait (per Tait flow rules) less available
                                # at BT, as just computed
                                TBTait3[i,3] = TaitHCP3[i,6] - BTHCP3[i,3]
            
                            elif Felton[i-1,3] == 114:
                                Felton[i,3] = 99
                                Felton[i,4] = 0
                                # BT available is BT unimpaired less Felton bypass
                                BTHCP3[i,3] = TaitHCP3[i,6]
                                Felton[i,5] = BTHCP3[i,3]
                                Felton[i,8] = TaitHCP3[i,6]
                                # When bypassing 40 cfs, available at Tait equals total 
                                # available at Tait (per Tait flow rules) less available
                                # at BT, as just computed
                                TBTait3[i,3] = TaitHCP3[i,6] - BTHCP3[i,3]
            
                            elif Felton[i-1,3] == 99:
                                Felton[i,3] = 99
                                Felton[i,4] = 0
                                # BT available is BT unimpaired less Felton bypass
                                BTHCP3[i,3] = TaitHCP3[i,6]
                                Felton[i,5] = BTHCP3[i,3]
                                Felton[i,8] = TaitHCP3[i,6]
                                # When bypassing 40 cfs, available at Tait equals total 
                                # available at Tait (per Tait flow rules) less available
                                # at BT, as just computed
                                TBTait3[i,3] = TaitHCP3[i,6] - BTHCP3[i,3]
    
                        else:
    
                            if BTUn[i,3] > TaitHCP3[i,6]:
    
                                BTHCP3[i,3] = TaitHCP3[i,6]
                                TBTait3[i,3] = TaitHCP3[i,6] - BTHCP3[i,3]
                                Felton[i,5] = BTHCP3[i,3]
                                Felton[i,8] = TaitHCP3[i,6]
    
                            else:
    
                                BTHCP3[i,3] = BTUn[i,3]
                                TBTait3[i,3] = TaitHCP3[i,6] - BTUn[i,3]
                                Felton[i,5] = BTHCP3[i,3]
                                Felton[i,8] = TaitHCP3[i,6]- BTUn[i,3]
                                
                else:
        
                    # Available flow equals zero.
                    BTHCP3[i,3] = 0
                    # The 5 CDFG potential production floor is always in effect
                    # BTHCP3[i,3] = BTUn[i,3] * 0.05
            
                
            # Section 3.6.2 Laguna
            
            # --- Tier III
            
                if LGAHCP3[i,3] > LGAHCP3[i,6]:
                    
                    if LGc[i,3] > LGAc[i,3] - LGAHCP3[i,6]:
                    
                        LGHCP3[i,3] = LGAc[i,3] - LGAHCP3[i,6]
                    
                    else:
                        
                        LGHCP3[i,3] = LGc[i,3]
                    
                else:
                    
                    LGHCP3[i,3] = 0
                    # LGHCP3[i,3] = LGc[i,3]*0.05
                    # 5-percent CDFG potential production floor
    
                if LGHCP3[i,3] < 0:
                    LGHCP3[i,3] = 0
                    # LGHCP3[i,3] = LGc[i,3]*0.05
                    # 5-percent CDFG potential production floor
    
            # Section 3.6.3 Majors
    
            # --- Majors, Tier III
                
                if MJAHCP3[i,3] > MJAHCP3[i,6]:
                    
                    if MJc[i,3] > MJAc[i,3]-MJAHCP3[i,6]:
                    
                        MJHCP3[i,3] = MJAc[i,3]-MJAHCP3[i,6]
                    
                    else:
                        MJHCP3[i,3] = MJc[i,3]
                    
                else:
                    
                    MJHCP3[i,3] = 0
                    # MJHCP3(i,4)=MJc(i,4)*0.05
                    # 5-percent CDFG potential production floor
    
                if MJHCP3[i,3] < 0:
                    
                    MJHCP3[i,3] = 0
                    # MJHCP3(i,4)=MJc(i,4)*0.05
                    # 5-percent CDFG potential production floor
    
            # Section 3.6.4 Liddell
    
            # --- Liddell, Tier III
            
                if LDAHCP3[i,3] > LDAHCP3[i,6]:
            
                    if LDc[i,3] > LDAc[i,3] - LDAHCP3[i,6]:
                        
                        LDHCP3[i,3] = LDAc[i,3] - LDAHCP3[i,6]
                        
                    else:
                        
                        LDHCP3[i,3] = LDc[i,3]
    
                else:
                    
                    LDHCP3[i,3] = 0
                    # LDHCP3[i,3] = LDc[i,3]*0.05
                    # 5-percent CDFG potential production floor
    
                if LDHCP3[i,3] < 0:
                    
                    LDHCP3[i,3] = 0
                    # LDHCP3[i,3] = LDc[i,3]*0.05
                    # 5-percent CDFG potential production floor
    
            ##### Section 3.7 Big Trees Pre-Existing (PX) leagal bypass requirement
            # Here I will calculate the available flow under the PX rules. The actual
            # available flow will be the smaller of the PX and HCP available flows.
            # The larger bypass requirement is used, therefore the smaller available
            # is used.
    
            BTPX = np.zeros((BTlength,6))
            BTPX[:,:3] = BT[:,:3]
            BTPX[:,3] = BT[:,3]
            BTPX[:,4] = BTPX[:,3] # BT Available flow under PX, start as BT(:,4)
            BTPX[:,5] = 0 # Will be the Legal Bypass requirement amount
            
            for i in range(BTlength):
                
                if BTPX[i,0] == 9:
                    
                    BTPX[i,4] = BTPX[i,3] - 10 # Calculate available flow after PX bypass
                    BTPX[i,5] = 10
                    
                elif BTPX[i,0] == 10:
                    
                    BTPX[i,4] = BTPX[i,3] - 25
                    BTPX[i,5] = 25
                    
                elif BTPX[i,0] == 11 or BTPX[i,0] == 12:
                    
                    BTPX[i,4] = BTPX[i,3] - 20
                    BTPX[i,5] = 20
                    
                elif BTPX[i,0] <= 5:
                
                    BTPX[i,4] = BTPX[i,3] - 20
                    BTPX[i,5] = 20
    
                if BTPX[i,4] < 0:
                    BTPX[i,4] = 0 # Prevent negative available flows.
    
            # Now choose if BTPX or BTHCP is to be used.
            # This will be different for each Tier.
            BTFinal3 = np.zeros((BTlength,4))
            BTFinal3[:,:3] = BT[:,:3]
            BTFinal3[:,3] = BTHCP3[:,3]
        
            for i in range(BTlength):
                
                if BTPX[i,4] < BTHCP3[i,3]:
                    if i == 9889:
                        print('heretait')
                    BTFinal3[i,3] = BTPX[i,4]
                    TBTait3[i,4] = BTHCP3[i,3] - BTPX[i,4]
    
            ##### Section 3.8 Calculate the "Trib*" flow
            # In this case Trib is in quotes because it isn't simply the tributary
            # flow. It will be the difference between the PX flow and the HCP@BT flow.
            # The goal is to account for the PX diversion at BT, but to make that flow
            # available for diversion at Tait, up to the HCP flow available.
            # TBTaitPX(:,1:3)=BT(:,1:3); Unimpaired by the City, but pre-HCP rules.
            # TBTait arrays were initialized earlier in the code, when transfering the
            # impact of the HCP rules from Tait upstream to Big Trees.
            
            TBTait3[:,5] = TBTait3[:,3] + TBTait3[:,4]
            
            ##### Section 3.9 Write the data to file:
            
            # Make a new Folder named for the input climate file name
            WriteDirName = './Output_HISTORICAL_SIM_Python/'
            str0 = [WriteDirName, WriteFileName]
            newStr = ''.join(str0)
    
            # Check to see if main output directory exists; if not, create it
            if not os.path.exists(WriteDirName):
                os.mkdir(WriteDirName)
                print('Output folder for all runs created')
            
            # Check to see if output directory for the run exists; if not, create it
            if not os.path.exists(newStr):
                os.mkdir(newStr)
                print('Output folder for current run created')
            
            # Data arrays will be written to output files. 
            
            # Analog date array
            str3 = '/Analog_Year.txt'
            name = ''.join([newStr, str3])
            with open(name, 'w') as f: # Write a text file for QA/QC
                np.savetxt(f, AnalogYr, fmt='%d') # All in one column format        
            
            # Monthly Hydrologic Condition
            name = ''.join([newStr, '/HCMon.txt'])
            with open(name, 'w') as f: # Write a text file for QA/QC
                np.savetxt(f, HydroCond_Mon, fmt='%f') # All in one column format
            
            # Annual Hydrologic Condition
            name = ''.join([newStr, '/HCAnn.txt'])
            with open(name, 'w') as f: # Write a text file for QA/QC
                np.savetxt(f, HydroCond_Ann, fmt='%f') # All in one column format
            
            # Annual Hydrologic Condition
            name = ''.join([newStr, '/HCHgraph.txt'])
            with open(name, 'w') as f: # Write a text file for QA/QC
                np.savetxt(f, HC_Hgraph, fmt='%f') # All in one column format
            
            # Big Trees Natural Flows
            name = ''.join([newStr, '/BTUnxl.txt'])
            with open(name, 'w') as f: # Write a text file for QA/QC
                np.savetxt(f, BTUn, fmt='%f') # All in one column format
            
            # San Vicente Natural Flows
            name = ''.join([newStr, '/SVxl.txt'])
            with open(name, 'w') as f: # Write a text file for QA/QC
                np.savetxt(f, SVc, fmt='%f') # All in one column format
            
            # Upper Laguna Natural Flows
            name = ''.join([newStr, '/LGxl.txt'])
            with open(name, 'w') as f: # Write a text file for QA/QC
                np.savetxt(f, LGc, fmt='%f') # All in one column format
            
            #Upper Majors Natural Flows
            name = ''.join([newStr, '/MJxl.txt'])
            with open(name, 'w') as f: # Write a text file for QA/QC
                np.savetxt(f, MJc, fmt='%f') # All in one column format
            
            # Upper Liddell Natural Flows
            name = ''.join([newStr, '/LDxl.txt'])
            with open(name, 'w') as f: # Write a text file for QA/QC
                np.savetxt(f, LDc, fmt='%f') # All in one column format
            
            # Loch Lomond Natural Flows
            name = ''.join([newStr, '/LLxl.txt'])
            with open(name, 'w') as f: # Write a text file for QA/QC
                np.savetxt(f, LLc, fmt='%f') # All in one column format
            
            ## ---------- Write data for anadromous stations, unimpared ----------------
            # Write data to file(s)
            # Confluence Formatting is not neccisary because the anadromous stations
            # are not nodes in Confluence. Only the columnar format is needed for 
            # these stations.
            
            # Laguna Anadromous station, combined modeled and gaged.
            name = ''.join([newStr, '/LGAc.txt'])
            with open(name, 'w') as f: # Write a text file for QA/QC
                np.savetxt(f, LGAc, fmt='%f') # All in one column format
            
            # Majors Anadromous station, combined modeled and gaged.
            name = ''.join([newStr, '/MJAc.txt'])
            with open(name, 'w') as f: # Write a text file for QA/QC
                np.savetxt(f, MJAc, fmt='%f') # All in one column format
            
            # Liddell Anadromous station, combined modeled and gaged.
            name = ''.join([newStr, '/LDAc.txt'])
            with open(name, 'w') as f: # Write a text file for QA/QC
                np.savetxt(f, LDAc, fmt='%f') # All in one column format
            
            # Newell Anadromous station, modeled (no gaged.)
            name = ''.join([newStr, '/NLAm.txt'])
            with open(name, 'w') as f: # Write a text file for QA/QC
                np.savetxt(f, NLAm, fmt='%f') # All in one column format
            
            # San Lorenzo Anadromous station (Tait St.), modeled only as of 7/19/2010.
            name = ''.join([newStr, '/Tc.txt'])
            with open(name, 'w') as f: # Write a text file for QA/QC
                np.savetxt(f, Tc, fmt='%f') # All in one column format
            
            # Watershed Contributions between Upper and Anadromous stations
            # Laguna Creek
            LGtrib = np.zeros((len(LGAc),4))
            LGtrib[:, 0:3] = LGAc[:, 0:3]
            LGtrib[:, 3] = LGAc[:, 3] - LGc[:, 3]
            
            name = ''.join([newStr, '/LGTrib.txt'])
            with open(name, 'w') as f: # Write a text file for QA/QC
                np.savetxt(f, LGtrib, fmt='%f') # Confluence Format
            
            # Majors Creek
            MJtrib = np.zeros((len(MJAc),4))
            MJtrib[:, 0:3] = MJAc[:, 0:3]
            MJtrib[:, 3] = MJAc[:, 3] - MJc[:, 3]
            
            name = ''.join([newStr, '/MJTrib.txt'])
            with open(name, 'w') as f: # Write a text file for QA/QC
                np.savetxt(f, MJtrib, fmt='%f') # Confluence Format
            
            # Liddell Creek
            LDtrib = np.zeros((len(LDAc),4))
            LDtrib[:, 0:3] = LDAc[:, 0:3]
            LDtrib[:, 3] = LDAc[:, 3] - LDc[:, 3]
            
            name = ''.join([newStr, '/LDTrib.txt'])
            with open(name, 'w') as f: # Write a text file for QA/QC
                np.savetxt(f, LDtrib, fmt='%f') # Confluence Format
            
            # Newell Creek
            NLAtrib = np.zeros((len(NLAm),4))
            NLAtrib[:, 0:3] = NLAm[:, 0:3]
            NLAtrib[:, 3] = NLAm[:, 3] - LLc[:, 3]
            
            name = ''.join([newStr, '/NLATrib.txt'])
            with open(name, 'w') as f: # Write a text file for QA/QC
                np.savetxt(f, NLAtrib, fmt='%f') # Confluence Format
            
            # Lower San Lorenzo River
            TAtrib = np.zeros((BTlength,4))
            TAtrib[:, 0:3] = BT[:, 0:3]
            TAtrib[:, 3] = Tc[:, 3]
            for i in range(BTlength):
                if Tc[i, 3] - BT[i, 3] > 0:
                    TAtrib[i, 3] = Tc[i, 3] - BT[i, 3]
                else:
                    TAtrib[i, 3] = 0
            
            name = ''.join([newStr, '/TAtrib.txt'])
            with open(name, 'w') as f: # Write a text file for QA/QC
                np.savetxt(f, TAtrib, fmt='%f') # All in one column format
            
            ## Section 3.5 Write data for anadromous stations, HCP flows 
            # Write data to file(s)
            # Confluence Formatting is not neccisary because the anadromous stations
            # are not nodes in Confluence. Only the columnar format is needed for 
            # these stations.
            
            # Laguna Anadromous station, HCP remaining for production flows
            name = ''.join([newStr, '/LGAHCP3.txt'])
            with open(name, 'w') as f: # Write a text file for QA/QC
                np.savetxt(f, LGAHCP3, fmt='%f') # All in one column format
            
            # Majors Anadromous station, HCP remaining for production flows
            name = ''.join([newStr, '/MJAHCP3.txt'])
            with open(name, 'w') as f: # Write a text file for QA/QC
                np.savetxt(f, MJAHCP3, fmt='%f') # All in one column format
            
            # Liddell Anadromous station, HCP remaining for production flows
            name = ''.join([newStr, '/LDAHCP3.txt'])
            with open(name, 'w') as f: # Write a text file for QA/QC
                np.savetxt(f, LDAHCP3, fmt='%f') # All in one column format
            
            # San Lorenzo at Tait St. station, HCP remaining for production flows  ##### SHAWN: THIS FILE SAVES TWICE?  OVERWRITTEN BELOW
            name = ''.join([newStr, '/TaitHCP3.txt'])
            with open(name, 'w') as f: # Write a text file for QA/QC
                np.savetxt(f, TaitHCP3, fmt='%f') # All in one column format
            
            # --- BT PreExisting Rules
            # Big Trees Confluence Formatting
            BTpx = np.zeros((1, 3))
            BTpx[0, 0] = BTPX[0, 2]
            BTpx[0, 1] = BTPX[0, 0]
            BTpx[0, 2] = BTPX[0, 4]
            
            name = ''.join([newStr, '/BTPX.txt'])
            with open(name, 'w') as f: # Write a text file for QA/QC
                np.savetxt(f, BTPX, fmt='%f') # All in one column format
            
            # --- BT Tier III
            # Big Trees Confluence Formatting
            BThcp3 = np.zeros((1, 3))
            BThcp3[0, 0] = BTFinal3[0, 2]
            BThcp3[0, 1] = BTFinal3[0, 0]
            BThcp3[0, 2] = BTFinal3[0, 3]
            
            name = ''.join([newStr, '/BTHCP3.txt'])
            with open(name, 'w') as f: # Write a text file for QA/QC
                np.savetxt(f, BTFinal3, fmt='%f') # All in one column format
            
            name = ''.join([newStr, '/Felton.txt'])
            with open(name, 'w') as f: # Write a text file for QA/QC
                np.savetxt(f, Felton, fmt='%f') # All in one column format
            
            # --- TB Tier III
            # Additional flow below Big Trees Confluence Formatting
            TB3 = np.zeros((1, 3))
            TB3[0, 0] = TBTait3[0, 2]
            TB3[0, 1] = TBTait3[0, 0]
            TB3[0, 2] = TBTait3[0, 5]
            
            name = ''.join([newStr, '/TaitHCP3.txt'])
            with open(name, 'w') as f: # Write a text file for QA/QC
                np.savetxt(f, TBTait3, fmt='%f') # All in one column format
            # -------------------------------------------------------------------------
            
            # -- Laguna Tier III
            LGhcp3 = np.zeros((1, 3))
            LGhcp3[0, 0] = LGHCP3[0, 2]
            LGhcp3[0, 1] = LGHCP3[0, 0] 
            LGhcp3[0, 2] = LGHCP3[0, 3]
            
            name = ''.join([newStr, '/LGHCP3.txt'])
            with open(name, 'w') as f: # Write a text file for QA/QC
                np.savetxt(f, LGHCP3, fmt='%f') # All in one column format
            
            # -------------------------------------------------------------------------
            
            # -- Majors, Tier III
            MJhcp3 = np.zeros((1, 3))
            MJhcp3[0, 0] = MJHCP3[0, 2]
            MJhcp3[0, 1] = MJHCP3[0, 0]
            MJhcp3[0, 2] = MJHCP3[0, 3]
            
            name = ''.join([newStr, '/MJHCP3.txt'])
            with open(name, 'w') as f: # Write a text file for QA/QC
                np.savetxt(f, MJHCP3, fmt='%f') # All in one column format
            
            # -------------------------------------------------------------------------
            
            # -- Liddell Springflow, Tier III
            LDhcp3 = np.zeros((1, 3))
            LDhcp3[0, 0] = LDHCP3[0, 2]
            LDhcp3[0, 1] = LDHCP3[0, 0]
            LDhcp3[0, 2] = LDHCP3[0, 3]
            
            name = ''.join([newStr, '/LDHCP3.txt'])
            with open(name, 'w') as f: # Write a text file for QA/QC
                np.savetxt(f, LDHCP3, fmt='%f') # All in one column format
    
            # -------------------------------------------------------------------------
            # Save the flow figure to file
            # name = strcat(newStr,'/BigTreesFlow.png');     
            # plt.savefig(name, dpi = 300, bbox_inches = 'tight')
    
            # -------------------------------------------------------------------------
            
            # Update the progress bar
            progress_bar.update(1)
            if kk == steps - 1:
                progress_bar.close()
                print('Simulations ran successfully')
            else:
                print(f'Just finished iteration {track}')
                track += 1